<a href="https://colab.research.google.com/github/Anunineleaps6/Capstone_Sales/blob/main/Copy_of_Capstone_sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install gspread oauth2client pandas numpy


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("/content/drive/MyDrive/CAPSTONE_FINAL/creds.json", scope)
client = gspread.authorize(creds)


In [6]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta


In [7]:
!pip install phonenumbers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 26.1 MB/s eta 0:00:00


In [8]:
import phonenumbers

def generate_phone(country, region_mapping):
    # Try to get the country code from the region_mapping first
    code = None
    for region, data in region_mapping.items():
        if country in data["countries"]:
            code = data["dial_code"].get(country)
            break

    if code:
        try:
            # Generate a random 10-digit number
            local_number = ''.join([str(random.randint(0, 9)) for _ in range(10)])
            # Attempt to parse and format using phonenumbers
            # Note: This might not create *valid* numbers, but will follow the format
            # We need a country code without the '+' for the parse function
            country_code_for_parse = code.lstrip('+')
            # We need to find a region code (like US, GB, IN) for the parse function
            # This is a simplified approach; a more robust solution would map countries to region codes
            country_region_code = ""
            for region_key, data in region_mapping.items():
                if country in data.get("countries", []):
                     # This is a simplification; need a proper country to region code mapping
                     # For now, we can try to infer from common country codes or use a default
                     # A more accurate approach would involve a comprehensive country code to region code mapping
                     # Let's use a placeholder logic or rely on the dial code if a direct mapping is complex
                     pass # Will refine this

            # A better approach might be to directly format a random number with the known dial code
            # We'll stick to generating a random 10-digit number and prefixing the dial code
            # as phonenumbers.parse requires a known region or a leading '+' which we already have

            # Let's refine the phonenumbers usage or simplify if direct generation is sufficient
            # Given the user wants the format "+[country_code]-[10_digits]", direct generation is simpler
            number = ''.join([str(random.randint(0, 9)) for _ in range(10)])
            return f"{code}-{number}"

        except Exception as e:
            print(f"Error generating phone for {country} with code {code}: {e}")
            # Fallback to a generic format if phonenumbers fails
            return f"{code}-{random.randint(1000000000, 9999999999)}"
    else:
        # Fallback for countries not in region_mapping
        return f"+00-{random.randint(1000000000, 9999999999)}"

# Now, modify the function call in the lead generation loop in cell 9212e2f3
# (This modification will be done in the next modify_cells command)

In [30]:
#LEADS
import pandas as pd
import random
import uuid
from datetime import datetime, timedelta
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# ---------- Google Sheets Authentication and Connection ----------
try:
    # Attempt to use the existing client object if defined
    google_sheets_client = client
except NameError:
    print("Google Sheets client not found. Attempting to authenticate and create a new client.")
    try:
        scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
        creds = ServiceAccountCredentials.from_json_keyfile_name("/content/drive/MyDrive/CAPSTONE_FINAL/creds.json", scope)
        google_sheets_client = gspread.authorize(creds)
        print("Successfully authenticated and created Google Sheets client.")
    except FileNotFoundError:
        print("Error: 'creds.json' not found. Please ensure your service account credentials file is in the correct path.")
        raise
    except Exception as e:
        print(f"An error occurred during Google Sheets authentication: {e}")
        raise

# Assuming 'spreadsheet' object is available from a previous cell that connected to Google Sheets
try:
    spreadsheet
except NameError:
    print("Spreadsheet object not found. Attempting to open spreadsheet.")
    try:
        spreadsheet = google_sheets_client.open("CRM_Capstone") # Replace with your spreadsheet name
        print("Successfully opened spreadsheet.")
    except Exception as e:
        print(f"An error occurred while opening the spreadsheet: {e}")
        raise

# 1. Read existing Leads data from Google Sheets
try:
    worksheet_leads = spreadsheet.worksheet("Leads")
    df_existing_leads = pd.DataFrame(worksheet_leads.get_all_records())
    print(f"Successfully loaded {len(df_existing_leads)} existing leads from Google Sheets.")
except gspread.WorksheetNotFound:
    print("Leads sheet not found. Starting with an empty DataFrame for existing leads.")
    df_existing_leads = pd.DataFrame(columns=[
        "Lead_ID", "Lead_Name", "Company_Name", "Source", "Region_ID", "Rep_ID",
        "Created_Date", "Current_Stage", "Email", "Phone", "Priority", "Last_Updated",
        "Deal_Source", "Company_ID", "Engagement_Level", "Follow_Up_Threshold",
        "Project_Term", "Budget_Tier", "Country", "Company_Industry", "Service_Category",
        "Deal_Status_Category" # Include this column as it might exist in the sheet
    ])
except Exception as e:
    print(f"An error occurred while loading existing leads: {e}")
    df_existing_leads = pd.DataFrame(columns=[
        "Lead_ID", "Lead_Name", "Company_Name", "Source", "Region_ID", "Rep_ID",
        "Created_Date", "Current_Stage", "Email", "Phone", "Priority", "Last_Updated",
        "Deal_Source", "Company_ID", "Engagement_Level", "Follow_Up_Threshold",
        "Project_Term", "Budget_Tier", "Country", "Company_Industry", "Service_Category",
        "Deal_Status_Category"
    ])


# ------------------------------------------
# 2. Region, Country, Dial Code Mapping (Used for phone number generation)
# ------------------------------------------
region_mapping = {
    "NA": {"countries": ["USA", "Canada"], "dial_code": {"USA": "+1", "Canada": "+1"}, "industries": ["Technology", "Healthcare", "Finance", "Retail"], "services": ["Data", "Marketing", "Cloud"]},
    "EU": {"countries": ["UK", "Germany", "France", "Netherlands", "Switzerland", "Sweden", "Israel"], "dial_code": {"UK": "+44", "Germany": "+49", "France": "+33", "Netherlands": "+31", "Switzerland": "+41", "Sweden": "+46", "Israel": "+972"}, "industries": ["Automotive", "Pharmaceuticals", "Banking", "Consulting"], "services": ["Consulting", "Engineering", "Operations"]},
    "APAC": {"countries": ["India", "Japan", "Singapore", "China", "Indonesia", "Australia", "New Zealand", "S. Korea"], "dial_code": {"India": "+91", "Japan": "+81", "Singapore": "+65", "China": "+86", "Indonesia": "+62", "Australia": "+61", "New Zealand": "+64", "S. Korea": "+82"}, "industries": ["Technology", "E-Commerce", "Manufacturing", "Finance"], "services": ["AI", "Staffing", "T&M"]},
    "ME": {"countries": ["UAE", "Saudi Arabia", "Qatar", "Bahrain"], "dial_code": {"UAE": "+971", "Saudi Arabia": "+966", "Qatar": "+974", "Bahrain": "+973"}, "industries": ["Energy", "Real Estate", "Banking"], "services": ["Data", "Consulting"]},
    "LA": {"countries": ["Brazil", "Mexico", "Argentina", "Chile", "Colombia", "Peru"], "dial_code": {"Brazil": "+55", "Mexico": "+52", "Argentina": "+54", "Chile": "+56", "Colombia": "+57", "Peru": "+51"}, "industries": ["Manufacturing", "Agriculture", "Energy"], "services": ["Operations", "Logistics"]},
    "AF": {"countries": ["S. Africa", "Mozambique", "Kenya", "Morocco"], "dial_code": {"S. Africa": "+27", "Mozambique": "+258", "Kenya": "+254", "Morocco": "+212"}, "industries": ["Telecommunications", "Mining", "Finance"], "services": ["Staffing", "Service"]}
}

# ------------------------------------------
# 3. Predefined Lead Data (for Company_Name, Country, Region, Lead_ID) - 108 Companies
# ------------------------------------------
predefined_leads = [
    # North America (20)
    ("Uber", "USA", "NA", "LDUBR-NA-001"), ("Walmart", "USA", "NA", "LDWMT-NA-002"), ("Ford Motor Co", "USA", "NA", "LDFRD-NA-003"), ("Amazon", "USA", "NA", "LDAMZ-NA-004"),
    ("Microsoft", "USA", "NA", "LDMST-NA-005"), ("Target", "USA", "NA", "LDTGT-NA-006"), ("Uber Eats", "USA", "NA", "LDUBE-NA-007"), ("CVS Health", "USA", "NA", "LDCVS-NA-008"),
    ("McKesson", "USA", "NA", "LDMCK-NA-009"), ("TD Bank Group", "Canada", "NA", "LDTDB-NA-010"), ("Shopify", "Canada", "NA", "LDSHF-NA-011"), ("Suncor Energy", "Canada", "NA", "LDSCR-NA-012"),
    ("Kaiser Permanente", "USA", "NA", "LDKPR-NA-013"), ("Wells Fargo", "USA", "NA", "LDWLF-NA-014"), ("Canadian Tire", "Canada", "NA", "LDCTC-NA-015"), ("American Tower", "USA", "NA", "LDAMT-NA-016"),
    ("Tim Hortons", "Canada", "NA", "LDTHC-NA-017"), ("Lowe's", "USA", "NA", "LDLWS-NA-018"), ("PicnicHealth", "USA", "NA", "LDPCH-NA-019"), ("Credit Saison India (USA operations)", "USA", "NA", "LDCSI-NA-020"),

    # Europe (16)
    ("Tesco", "UK", "EU", "LDTSCO-EU-021"), ("Lufthansa Group", "Germany", "EU", "LDLFT-EU-022"), ("Allianz Direct", "Germany", "EU", "LDALZ-EU-023"), ("HSBC", "UK", "EU", "LDHSBC-EU-024"),
    ("Sage Group", "UK", "EU", "LDSGE-EU-025"), ("Schneider Electric", "France", "EU", "LDSNE-EU-026"), ("Philips Healthcare", "Netherlands", "EU", "LDPHL-EU-027"), ("Vodafone", "UK", "EU", "LDVDF-EU-028"),
    ("Siemens", "Germany", "EU", "LDSMN-EU-029"), ("Nestlé", "Switzerland", "EU", "LDNST-EU-030"), ("ING Group", "Netherlands", "EU", "LDING-EU-031"), ("ABB", "Switzerland", "EU", "LDABB-EU-032"),
    ("Volvo Group", "Sweden", "EU", "LDVLV-EU-033"), ("Carrefour", "France", "EU", "LDCFR-EU-034"), ("ZF Group", "Germany", "EU", "LDZFG-EU-035"), ("Trigo", "UK", "EU", "LDTRG-EU-036"), # Assuming UK operations for Trigo

    # Asia-Pacific (14)
    ("DBS Bank", "Singapore", "APAC", "LDDBS-APAC-037"), ("Kiwibank", "New Zealand", "APAC", "LDKWB-APAC-038"), ("Reliance Retail", "India", "APAC", "LDRLR-APAC-039"), ("Tata Consultancy Services", "India", "APAC", "LDTCS-APAC-040"),
    ("SoftBank", "Japan", "APAC", "LDSFB-APAC-041"), ("Tokopedia / GoTo", "Indonesia", "APAC", "LDTKP-APAC-042"), ("Telstra", "Australia", "APAC", "LDTSL-APAC-043"), ("Westpac", "Australia", "APAC", "LDWSP-APAC-044"),
    ("infosys", "India", "APAC", "LDINF-APAC-045"), ("Grab", "Singapore", "APAC", "LDGRB-APAC-046"), ("JioMart", "India", "APAC", "LDJMR-APAC-047"), ("Toyota", "Japan", "APAC", "LDTYT-APAC-048"),
    ("Sony", "Japan", "APAC", "LDSNY-APAC-049"), ("Samsung", "S. Korea", "APAC", "LDSMS-APAC-050"),

    # Middle East (4)
    ("Saudi Aramco", "Saudi Arabia", "ME", "LDSAR-ME-051"), ("ADNOC", "UAE", "ME", "LDADC-ME-052"), ("QatarEnergy", "Qatar", "ME", "LDQTE-ME-053"), ("Investcorp", "Bahrain", "ME", "LDINV-ME-054"),

    # Latin America (10)
    ("Petrobras", "Brazil", "LA", "LDPBR-LA-055"), ("América Móvil", "Mexico", "LA", "LDAMX-LA-056"), ("MercadoLibre", "Argentina", "LA", "LDMCL-LA-057"), ("Codelco", "Chile", "LA", "LDCDL-LA-058"),
    ("Ecopetrol", "Colombia", "LA", "LDECP-LA-059"), ("Latam Airlines", "Chile", "LA", "LDLTM-LA-060"), ("Natura", "Brazil", "LA", "LDNTR-LA-061"), ("Itau Unibanco", "Brazil", "LA", "LDITU-LA-062"),
    ("Bancolombia", "Colombia", "LA", "LDBCL-LA-063"), ("Grupo Mexico", "Mexico", "LA", "LDGPM-LA-064"),

    # Africa (12)
    ("MTN Group", "S. Africa", "AF", "LDMTN-AF-065"), ("Sasol", "S. Africa", "AF", "LDSAL-AF-066"), ("Standard Bank Group", "S. Africa", "AF", "LDSBG-AF-067"), ("Vodacom", "S. Africa", "AF", "LDVDC-AF-068"),
    ("Safaricom", "Kenya", "AF", "LDSFC-AF-069"), ("KCB Group", "Kenya", "AF", "LDKCB-AF-070"), ("Attijariwafa Bank", "Morocco", "AF", "LDATJ-AF-071"), ("FirstRand", "S. Africa", "AF", "LDFRD-AF-072"),
    ("Absa Group", "S. Africa", "AF", "LDABG-AF-073"), ("Nedbank Group", "S. Africa", "AF", "LDNDB-AF-074"), ("Equity Group Holdings", "Kenya", "AF", "LDEGH-AF-075"), ("Telkom", "S. Africa", "AF", "LDTLK-AF-076"),

    # Adding more predefined companies to reach 108, maintaining the pattern and region assignment
    ("Infosys", "India", "APAC", "LDINF-APAC-077"), ("Wipro", "India", "APAC", "LDWIP-APAC-078"), ("HCL Technologies", "India", "APAC", "LDHCL-APAC-079"),
    ("Tech Mahindra", "India", "APAC", "LDTMH-APAC-080"), ("Larsen & Toubro Infotech", "India", "APAC", "LDLTI-APAC-081"), ("Mindtree", "India", "APAC", "LDMNT-APAC-082"),
    ("Mphasis", "India", "APAC", "LDMPH-APAC-083"), ("L&T Technology Services", "India", "APAC", "LDLTTS-APAC-084"), ("Persistent Systems", "India", "APAC", "LDPST-APAC-085"),
    ("Coforge", "India", "APAC", "LDCFG-APAC-086"), ("Zensar Technologies", "India", "APAC", "LDZNR-APAC-087"), ("Firstsource Solutions", "India", "APAC", "LDFRS-APAC-088"),
    ("ExlService Holdings", "India", "APAC", "LDEXL-APAC-089"), ("Genus Power Infrastructures", "India", "APAC", "LDGPI-APAC-090"), ("Cyient", "India", "APAC", "LDCYI-APAC-091"),

    ("Tata Motors", "India", "APAC", "LDTMO-APAC-092"), ("Mahindra & Mahindra", "India", "APAC", "LDMAM-APAC-093"), ("Maruti Suzuki India", "India", "APAC", "LDMSI-APAC-094"),
    ("Hyundai Motor India", "India", "APAC", "LDHMI-APAC-095"), ("Kia India", "India", "APAC", "LDKIA-APAC-096"), ("Toyota Kirloskar Motor", "India", "APAC", "LDTKM-APAC-097"),
    ("Honda Cars India", "India", "APAC", "LDHCI-APAC-098"), ("Renault India", "India", "APAC", "LDRNI-099"), ("Nissan Motor India", "India", "APAC", "LDNMI-APAC-100"),

    ("Apple", "USA", "NA", "LDAPL-NA-101"), ("Google", "USA", "NA", "LDGOO-NA-102"), ("Facebook", "USA", "NA", "LDFAB-NA-103"), ("Netflix", "USA", "NA", "LDNFL-NA-104"),
    ("Salesforce", "USA", "NA", "LDSFC-NA-105"), ("Tesla", "USA", "NA", "LDTES-NA-106"), ("IBM", "USA", "NA", "LDIBM-NA-107"), ("Intel", "USA", "NA", "LDINT-NA-108")
]

num_predefined_leads = len(predefined_leads)

# Get existing Lead IDs
existing_lead_ids = df_existing_leads['Lead_ID'].tolist()

# ------------------------------------------
# 3. Generate Sales Reps (Unevenly distributed based on region size)
# ------------------------------------------
# Re-read or ensure rep_df is available
try:
    rep_df
except NameError:
     # If rep_df is not defined, regenerate it (or read from sheet if available)
     print("rep_df not found. Regenerating Sales Reps.")
     rep_names = [
        "Alice Smith", "Bob Johnson", "Charlie Lee", "Diana King", "Edward Brown",
        "Fiona Green", "George White", "Hannah Black", "Ian Gray", "Jill Lewis",
        "Kevin Martin", "Laura Scott", "Mike Turner", "Nina Adams", "Oscar Clark",
        "Paula Young", "Quinn Hall", "Rachel Allen", "Steve Wright", "Tina Baker"
    ]
     region_rep_distribution = {"NA": 4, "EU": 4, "APAC": 5, "ME": 2, "LA": 3, "AF": 2}
     rep_list = []
     rep_name_index = 0
     for region, count in region_rep_distribution.items():
        for i in range(count):
            full_name = rep_names[rep_name_index]
            initials = "".join([part[0] for part in full_name.split()]).upper()
            rep_id = f"{initials}_{region}"
            rep_list.append({"rep_id": rep_id, "rep_name": full_name, "region": region})
            rep_name_index += 1
     rep_df = pd.DataFrame(rep_list)
     print("Sales Reps regenerated.")


# ------------------------------------------
# 4. Generate Leads DataFrame
# ------------------------------------------
def get_company_email(name):
    # Basic email domain generation
    domain = name.lower().replace(" ", "").replace("&", "").replace(".", "").replace("/", "").replace("(", "").replace(")", "") + ".com"
    return domain

# Use the generate_phone function defined in the previous cell (MCLKkiJr7tQA)

def generate_timestamp(start_days_ago=730): # Up to 2 years ago
    created = datetime.now() - timedelta(days=random.randint(0, start_days_ago), minutes=random.randint(0, 1440), seconds=random.randint(0, 86400))
    min_buffer_days = 1
    last_updated = created + timedelta(days=random.randint(min_buffer_days, 60), minutes=random.randint(0, 1440), seconds=random.randint(0, 86400))
    # Ensure last_updated is not in the future
    if last_updated > datetime.now():
        last_updated = datetime.now() - timedelta(minutes=random.randint(1, 60)) # Set to recent past

    return created.strftime("%Y-%m-%d %H:%M:%S"), last_updated.strftime("%Y-%m-%d %H:%M:%S")

def get_company_initials(name):
    return "".join([w[0] for w in name.split()]).upper()

all_service_categories = list(set([service for region_data in region_mapping.values() for service in region_data.get('services', [])]))
all_industries = list(set([industry for region_data in region_mapping.values() for industry in region_data.get('industries', [])]))

company_name_parts = {
    "adjectives": ["Global", "Strategic", "Innovative", "Dynamic", "Future", "Leading", "Creative", "Advanced"],
    "nouns": ["Solutions", "Systems", "Analytics", "Consulting", "Tech", "Ventures", "Partners", "Industries"]
}

def generate_random_company_name(adjectives, nouns):
    return f"{random.choice(adjectives)}{random.choice(nouns)}"

def get_abbreviated_company_name(name):
    words = name.split()
    if len(words) > 1:
        return "".join([w[:2] for w in words]).upper()
    else:
        return name[:4].upper()


new_leads_data = []

funnel_stages = [
    "Lead Captured", "Qualified", "Contacted", "Responded", "Demo Scheduled",
    "Demo", "Demo Completed", "Proposal Sent", "Proposal", "Negotiation"
]

potential_early_loss_stages = ["Demo", "Demo Completed", "Proposal Sent", "Proposal"]
early_loss_criteria = {
    "Demo": {"engagement_level": ["Low"], "random_chance": 0.4},
    "Demo Completed": {"budget_tier": ["20-50L"], "engagement_level": ["Low"], "random_chance": 0.3},
    "Proposal": {"engagement_level": ["Low", "Medium"], "priority": ["Low"], "random_chance": 0.5}
}
negotiation_loss_criteria = {"budget_tier": ["20-50L"], "engagement_level": ["Low"], "random_chance": 0.6}

# Adjusted ongoing_percentage and total leads to generate *new* data
ongoing_percentage = 0.30
# Determine how many *new* leads to generate to reach the total target
total_leads_target = 2000 # Total desired leads (existing + new)
num_new_leads_to_generate = max(0, total_leads_target - len(df_existing_leads))

# Distribute ongoing status across *all* leads (existing and new)
# Calculate the number of ongoing leads needed in total
total_ongoing_leads_needed = int(total_leads_target * ongoing_percentage)

# Determine how many *new* ongoing leads need to be generated
# This is tricky because existing leads already have stages.
# For simplicity, let's generate new leads and assign stages randomly based on the target distribution,
# aiming for the overall ongoing percentage.

ongoing_stage_distribution = {
    "Lead Captured": 0.031, "Qualified": 0.031, "Contacted": 0.044, "Responded": 0.019,
    "Demo Scheduled": 0.025, "Demo": 0.006, "Demo Completed": 0.031, "Proposal Sent": 0.038,
    "Proposal": 0.044, "Negotiation": 0.031
}

# Create a list of stages for random selection for *new* ongoing leads
ongoing_stages_list = []
total_distribution_percentage = sum(ongoing_stage_distribution.values())
scaling_factor = num_new_leads_to_generate / total_distribution_percentage if total_distribution_percentage > 0 else 0

for stage, percentage in ongoing_stage_distribution.items():
    num_to_add = int(percentage * scaling_factor)
    ongoing_stages_list.extend([stage] * num_to_add)

# Shuffle the list for random assignment
random.shuffle(ongoing_stages_list)
ongoing_stage_index = 0


specific_sources = {
    "Inbound": [
        "Contact Us", "Demo Request", "Pricing Page", "Google", "Content Marketing",
        "Webinars", "LinkedIn", "Twitter", "Facebook", "Instagram posts",
        "Referrals from Existing Customers", "Word of Mouth"
    ],
    "Outbound": [
        "Cold Email Campaigns", "Cold Calling", "Telemarketing", "Google Ads",
        "LinkedIn Ads", "Facebook Ads", "Instagram Ads", "Twitter Ads",
        "Display Ads", "Retargeting Campaigns",
        "Third-Party Lead Generation Vendors", "Lists"
    ]
}

all_specific_inbound_sources = specific_sources["Inbound"]
all_specific_outbound_sources = specific_sources["Outbound"]

all_countries = [country for region_data in region_mapping.values() for country in data["countries"]]
all_regions = list(region_mapping.keys())

# Track generated lead IDs to ensure uniqueness among new leads and against existing
generated_lead_ids = set(df_existing_leads['Lead_ID'].tolist())

# Generate new leads
for i in range(num_new_leads_to_generate):
    company_name = generate_random_company_name(company_name_parts["adjectives"], company_name_parts["nouns"])
    country = random.choice(all_countries)
    # Find region for the selected country
    region = next(r for r, data in region_mapping.items() if country in data["countries"])

    company_abbreviation = get_abbreviated_company_name(company_name)
    # Generate a unique Lead_ID
    lead_id_counter = len(generated_lead_ids) + 1
    lead_id = f"LD{company_abbreviation}-{region}-{lead_id_counter:04}"
    # Ensure the generated ID is unique among all leads (existing + new)
    while lead_id in generated_lead_ids:
         lead_id_counter += 1
         lead_id = f"LD{company_abbreviation}-{region}-{lead_id_counter:04}"

    generated_lead_ids.add(lead_id)


    company_email_domain = get_company_email(company_name)
    company_id = f"CMP_{get_company_initials(company_name)}_{lead_id_counter:04}" # Use same counter for company ID

    created_date, last_updated = generate_timestamp()
    deal_source = random.choices(["Inbound", "Outbound"], weights=[0.4, 0.6])[0]

    if deal_source == "Inbound":
        source = random.choice(all_specific_inbound_sources)
    else:
        source = random.choice(all_specific_outbound_sources)

    phone = generate_phone(country, region_mapping)

    region_reps = rep_df[rep_df["region"] == region]
    assigned_rep_id = region_reps.sample(n=1).iloc[0]["rep_id"] if not region_reps.empty else "Unassigned"

    lead_email = f"contact_{random.randint(1,100)}@{company_email_domain}"

    current_region = None
    for r, data in region_mapping.items():
        if country in data["countries"]:
            current_region = r
            break

    service_category = random.choice(region_mapping.get(current_region, {}).get('services', all_service_categories)) if current_region and region_mapping.get(current_region, {}).get('services') else random.choice(all_service_categories)
    company_industry = random.choice(region_mapping.get(current_region, {}).get('industries', all_industries)) if current_region and region_mapping.get(current_region, {}).get('industries') else random.choice(all_industries)

    engagement_level = random.choice(["High", "Medium", "Low"])
    follow_up_threshold = random.choice([3, 5, 7, 10])
    project_term = random.choice(["0-6 months", "6-12 months", "12-24 months", ">24 months"])
    budget_tier = random.choice(["20-50L", "50-100L", "100L+"])
    priority = random.choice(["High", "Medium", "Low"])

    # Determine the final stage for the new lead
    # Aim for the overall ongoing percentage across all leads (existing + new)
    # If we still need more ongoing leads to meet the total target ongoing count
    # AND we have ongoing stages left to assign from the generated list
    total_current_leads = len(df_existing_leads) + len(new_leads_data) + 1 # Leads processed so far in this loop + existing
    current_ongoing_count = len(df_existing_leads[df_existing_leads['Current_Stage'].str.startswith('Ongoing')].index) + len([nl for nl in new_leads_data if nl['Current_Stage'].startswith('Ongoing')]) # Count ongoing leads generated so far
    remaining_ongoing_needed = max(0, total_ongoing_leads_needed - current_ongoing_count)
    remaining_new_leads_to_process = num_new_leads_to_generate - i - 1 # Leads remaining to generate in this loop


    # Assign ongoing if needed to meet the target percentage, prioritizing if we are below target
    assign_ongoing = False
    if remaining_new_leads_to_process > 0:
         # Simple heuristic: if the current ongoing percentage is below the target, have a higher chance of generating ongoing
         current_ongoing_percentage = (current_ongoing_count + 1) / total_current_leads if total_current_leads > 0 else 0
         if current_ongoing_percentage < ongoing_percentage:
             assign_ongoing = random.random() < (ongoing_percentage - current_ongoing_percentage) * 2 # Increase chance if below target
         else:
             assign_ongoing = random.random() < ongoing_percentage # Base chance if at or above target

         # Ensure we don't exceed the total target ongoing count
         if current_ongoing_count >= total_ongoing_leads_needed:
             assign_ongoing = False


    if assign_ongoing and ongoing_stages_list:
        # Assign an intermediate stage from the ongoing list
        final_stage = ongoing_stages_list[ongoing_stage_index % len(ongoing_stages_list)]
        ongoing_stage_index += 1
        final_stage = f"Ongoing - {final_stage}"
    else:
        # Simulate progression to Won/Lost
        final_stage = "Lead Captured"
        for stage in funnel_stages:
            if stage == "Lead Captured":
                pass
            elif stage in potential_early_loss_stages:
                met_early_loss_criteria = False
                if stage in early_loss_criteria:
                    criteria = early_loss_criteria[stage]
                    budget_match = budget_tier in criteria.get("budget_tier", [budget_tier])
                    engagement_match = engagement_level in criteria.get("engagement_level", [engagement_level])
                    priority_match = priority in criteria.get("priority", [priority])
                    random_chance_met = random.random() < criteria.get("random_chance", 0)
                    if budget_match and engagement_match and priority_match and random_chance_met:
                        met_early_loss_criteria = True
                if met_early_loss_criteria:
                    final_stage = "Deal Lost"
                    break
                else:
                    final_stage = stage
            elif stage == "Negotiation":
                final_stage = stage
            if final_stage == "Negotiation":
                met_negotiation_loss_criteria = False
                criteria = negotiation_loss_criteria
                budget_match = budget_tier in criteria.get("budget_tier", [budget_tier])
                engagement_match = engagement_level in criteria.get("engagement_level", [engagement_level])
                random_chance_met = random.random() < criteria.get("random_chance", 0)
                if budget_match and engagement_match and random_chance_met:
                    final_stage = "Deal Lost"
                else:
                    final_stage = "Deal Won"
                break
            else:
                final_stage = stage


    new_leads_data.append({
        "Lead_ID": lead_id,
        "Lead_Name": f"{company_name} Lead",
        "Company_Name": company_name,
        "Source": source,
        "Region_ID": region,
        "Rep_ID": assigned_rep_id,
        "Created_Date": created_date,
        "Current_Stage": final_stage,
        "Email": lead_email,
        "Phone": phone,
        "Priority": priority,
        "Last_Updated": last_updated,
        "Deal_Source": deal_source,
        "Company_ID": company_id,
        "Engagement_Level": engagement_level,
        "Follow_Up_Threshold": follow_up_threshold,
        "Project_Term": project_term,
        "Budget_Tier": budget_tier,
        "Country": country,
        "Company_Industry": company_industry,
        "Service_Category": service_category
    })

# Combine existing and new leads
df_new_leads = pd.DataFrame(new_leads_data)
df_lead = pd.concat([df_existing_leads, df_new_leads], ignore_index=True)

# Recalculate Deal_Status_Category for the combined DataFrame
# This requires df_stage_history, which is generated in a later cell.
# For now, we will calculate based on the generated df_stage_history (which might be incomplete)
# The ETL cell will correctly recalculate this after loading all data.

# Temporarily handle the case where df_stage_history might not be fully populated yet
try:
    df_stage_history_for_categorization = df_stage_history
except NameError:
    print("Warning: df_stage_history not found during initial lead categorization. Categorizing ongoing deals based on Created_Date only.")
    df_stage_history_for_categorization = pd.DataFrame(columns=['Lead_ID', 'Stage', 'Timestamp'])


def categorize_deal_status(row, df_stage_history):
    if row['Current_Stage'] == 'Deal Won':
        return 'Deal Won'
    elif row['Current_Stage'] == 'Deal Lost':
        return 'Deal Lost'
    elif row['Current_Stage'].startswith('Ongoing - '):
        lead_id = row['Lead_ID']
        latest_ongoing_stage_timestamp = df_stage_history[
            (df_stage_history['Lead_ID'] == lead_id) &
            (df_stage_history['Stage'].isin(funnel_stages))
        ]['Timestamp'].max()

        # If no ongoing stage history found, use Created_Date
        if pd.isna(latest_ongoing_stage_timestamp):
             try:
                latest_ongoing_stage_timestamp = datetime.strptime(row['Created_Date'], "%Y-%m-%d %H:%M:%S")
             except (ValueError, TypeError):
                 # Handle cases where Created_Date might not be in expected format or is missing
                 return 'Ongoing - Cold' # Default to cold if date is invalid


        duration_since_last_update = datetime.now() - latest_ongoing_stage_timestamp
        two_months_in_days = 60

        if duration_since_last_update.days > two_months_in_days:
            return 'Ongoing - Cold'
        else:
            return 'Ongoing - Active'
    else:
        return 'Other'

# Apply the categorization to the combined dataframe
df_lead['Deal_Status_Category'] = df_lead.apply(
    lambda row: categorize_deal_status(row, df_stage_history_for_categorization),
    axis=1
)

# Ensure column order
df_lead = df_lead[[
    "Lead_ID", "Lead_Name", "Company_Name", "Source", "Region_ID", "Rep_ID",
    "Created_Date", "Current_Stage", "Email", "Phone", "Priority", "Last_Updated",
    "Deal_Source", "Company_ID", "Engagement_Level", "Follow_Up_Threshold",
    "Project_Term", "Budget_Tier", "Country", "Company_Industry", "Service_Category",
    "Deal_Status_Category" # Include the new column
]]


# ------------------------------------------
# 5. Export or View
# ------------------------------------------
# Save to CSV (optional, handled in ETL cell)
# df_lead.to_csv("crm_leads_generated.csv", index=False)

# Display sample
display(df_lead.head())
display(df_lead.tail())
print(f"Generated {len(df_new_leads)} new leads. Total leads: {len(df_lead)}")

# Display value counts
display(df_lead['Deal_Status_Category'].value_counts())
display(df_lead['Source'].value_counts())

Successfully loaded 2003 existing leads from Google Sheets.


,Lead_ID,Lead_Name,Company_Name,Source,Region_ID,Rep_ID,Created_Date,Current_Stage,Email,Phone,...,Deal_Source,Company_ID,Engagement_Level,Follow_Up_Threshold,Project_Term,Budget_Tier,Country,Company_Industry,Service_Category,Deal_Status_Category
0,LDUBR-NA-001,Uber Lead,Uber,Instagram Ads,NA,BJ_NA,2024-10-28 4:03:50,Deal Lost,contact_70@uber.com,-528015363,...,Outbound,CMP_U,Medium,7,>24 months,100L+,USA,Technology,Marketing,Deal Lost
1,LDWMT-NA-002,Walmart Lead,Walmart,Google,NA,DK_NA,2024-10-17 12:47:49,Deal Won,contact_61@walmart.com,-587713714,...,Inbound,CMP_W,High,5,6-12 months,50-100L,USA,Retail,Cloud,Deal Won
2,LDFRD-NA-003,Ford Motor Co Lead,Ford Motor Co,Google,NA,DK_NA,2024-04-20 15:52:48,Deal Won,contact_39@fordmotorco.com,-4240414113,...,Inbound,CMP_FMC,Low,7,0-6 months,20-50L,USA,Retail,Data,Deal Won
3,LDAMZ-NA-004,Amazon Lead,Amazon,LinkedIn Ads,NA,CL_NA,2025-02-23 12:08:37,Deal Won,contact_65@amazon.com,-295989703,...,Outbound,CMP_A,Medium,5,0-6 months,100L+,USA,Finance,Marketing,Deal Won
4,LDMST-NA-005,Microsoft Lead,Microsoft,Instagram Ads,NA,DK_NA,2024-05-30 15:44:04,Deal Lost,contact_77@microsoft.com,-6876180902,...,Outbound,CMP_M,Low,7,6-12 months,20-50L,USA,Healthcare,Cloud,Deal Lost


,Lead_ID,Lead_Name,Company_Name,Source,Region_ID,Rep_ID,Created_Date,Current_Stage,Email,Phone,...,Deal_Source,Company_ID,Engagement_Level,Follow_Up_Threshold,Project_Term,Budget_Tier,Country,Company_Industry,Service_Category,Deal_Status_Category
1998,LDINNO-NA-1999,InnovativeTech Lead,InnovativeTech,Cold Calling,NA,AS_NA,2024-12-08 8:12:20,Deal Won,contact_22@innovativetech.com,-3240368818,...,Outbound,CMP_I_1999,Medium,3,>24 months,20-50L,Canada,Healthcare,Marketing,Deal Won
1999,LDGLOB-NA-2000,GlobalIndustries Lead,GlobalIndustries,Referrals from Existing Customers,NA,DK_NA,2024-09-16 9:50:44,Deal Won,contact_89@globalindustries.com,-9218476218,...,Inbound,CMP_G_2000,High,7,>24 months,50-100L,Canada,Finance,Data,Deal Won
2000,LDINNO-NA-1997,GlobalConsulting Lead,GlobalConsulting,Google,NA,BJ_NA,2024-03-08 1:21:37,Deal Won,contact_81@globalconsulting.com,-3315598761,...,Inbound,CMP_G_1997,Medium,7,6-12 months,100L+,Canada,Retail,Marketing,Deal Won
2001,LDINNO-NA-1998,LeadingPartners Lead,LeadingPartners,LinkedIn,NA,BJ_NA,2024-02-20 17:13:52,Deal Lost,contact_11@leadingpartners.com,-1799890194,...,Inbound,CMP_L_1998,Medium,3,>24 months,50-100L,Canada,Retail,Marketing,Deal Lost
2002,LDINNO-NA-2000,GlobalIndustries Lead,GlobalIndustries,Referrals from Existing Customers,NA,DK_NA,2024-09-16 9:50:44,Deal Won,contact_89@globalindustries.com,-9218476218,...,Inbound,CMP_G_2000,High,7,>24 months,50-100L,Canada,Finance,Data,Deal Won


Generated 0 new leads. Total leads: 2003


,count
Deal_Status_Category,
Deal Won,1135
Ongoing - Cold,441
Deal Lost,359
Ongoing - Active,68


,count
Source,
LinkedIn Ads,111
Telemarketing,111
Google Ads,109
Twitter Ads,108
Instagram Ads,108
Retargeting Campaigns,101
Cold Calling,97
Third-Party Lead Generation Vendors,97
Display Ads,96


In [31]:
#COMPANIES
import pandas as pd
import numpy as np
import random
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# ---------- Google Sheets Authentication and Connection ----------
try:
    google_sheets_client = client
except NameError:
    print("Google Sheets client not found. Attempting to authenticate and create a new client.")
    try:
        scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
        creds = ServiceAccountCredentials.from_json_keyfile_name("/content/drive/MyDrive/CAPSTONE_FINAL/creds.json", scope)
        google_sheets_client = gspread.authorize(creds)
        print("Successfully authenticated and created Google Sheets client.")
    except FileNotFoundError:
        print("Error: 'creds.json' not found. Please ensure your service account credentials file is in the correct path.")
        raise
    except Exception as e:
        print(f"An error occurred during Google Sheets authentication: {e}")
        raise

try:
    spreadsheet
except NameError:
    print("Spreadsheet object not found. Attempting to open spreadsheet.")
    try:
        spreadsheet = google_sheets_client.open("CRM_Capstone") # Replace with your spreadsheet name
        print("Successfully opened spreadsheet.")
    except Exception as e:
        print(f"An error occurred while opening the spreadsheet: {e}")
        raise


# 1. Read existing Companies data from Google Sheets
try:
    worksheet_companies = spreadsheet.worksheet("Companies")
    df_existing_companies = pd.DataFrame(worksheet_companies.get_all_records())
    print(f"Successfully loaded {len(df_existing_companies)} existing companies from Google Sheets.")
except gspread.WorksheetNotFound:
    print("Companies sheet not found. Starting with an empty DataFrame for existing companies.")
    df_existing_companies = pd.DataFrame(columns=[
        "Company_ID", "Company_Name", "Company_Industry", "Company_Size",
        "Company_Ownership", "Company_Funding", "Country", "Interest_Area"
    ])
except Exception as e:
    print(f"An error occurred while loading existing companies: {e}")
    df_existing_companies = pd.DataFrame(columns=[
        "Company_ID", "Company_Name", "Company_Industry", "Company_Size",
        "Company_Ownership", "Company_Funding", "Country", "Interest_Area"
    ])


# 2. Define attributes for generating new companies
industries = ["Technology", "Healthcare", "Finance", "Manufacturing", "Retail", "E-Commerce", "Green Tech", "Mobility", "Real Estate", "Banking", "Logistics", "Housing", "Energy"]
sizes = ["Start-up", "Small business", "Medium Enterprise", "Large Enterprise"]
ownerships = ["Public Sector", "Private Sector", "Joint Venture", "Franchise"]
funding = ["Bootstrapped", "Seed", "Series A", "Series B", "IPO"]
interest_areas = ["Data", "Marketing", "Cloud", "Design", "HR", "AI", "Consulting", "Engineering", "Operations", "Staffing", "T&M"]

company_data = []
added_company_ids = set(df_existing_companies['Company_ID'].tolist()) # Get existing Company IDs

# Add companies from df_lead that are not already in the existing or new list
# Use unique Company_IDs from the combined df_lead
try:
    unique_company_ids_from_leads = df_lead['Company_ID'].unique().tolist()
except NameError:
    print("Error: df_lead not found. Cannot generate companies based on leads.")
    unique_company_ids_from_leads = [] # Empty list if df_lead is not available


for company_id in unique_company_ids_from_leads:
    if company_id not in added_company_ids:
        # Find the lead associated with this company_id to get company_name and country
        try:
            lead_info = df_lead[df_lead['Company_ID'] == company_id].iloc[0]
            company_name = lead_info['Company_Name']
            country = lead_info['Country']

            # Attempt to get industry and service from region_mapping if available, otherwise use random
            current_region_data = next((data for region, data in region_mapping.items() if country in data["countries"]), None)

            company_industry = random.choice(current_region_data.get('industries', industries)) if current_region_data and current_region_data.get('industries') else random.choice(industries)
            interest_area = random.choice(current_region_data.get('services', interest_areas)) if current_region_data and current_region_data.get('services') else random.choice(interest_areas)

            company_data.append({
                "Company_ID": company_id,
                "Company_Name": company_name,
                "Company_Industry": company_industry,
                "Company_Size": random.choice(sizes),
                "Company_Ownership": random.choice(ownerships),
                "Company_Funding": random.choice(funding),
                "Country": country,
                "Interest_Area": interest_area
            })
            added_company_ids.add(company_id)
        except Exception as e:
             print(f"Warning: Could not generate company data for Company_ID {company_id} from df_lead: {e}")
             # Skip this company if lead info cannot be retrieved


# Combine existing and new companies
df_new_companies = pd.DataFrame(company_data)
df_company = pd.concat([df_existing_companies, df_new_companies], ignore_index=True)

display(df_company.head())
display(df_company.tail())
print(f"Generated {len(df_new_companies)} new companies. Total companies: {len(df_company)}")

Successfully loaded 1963 existing companies from Google Sheets.


,Company_ID,Company_Name,Company_Industry,Company_Size,Company_Ownership,Company_Funding,Country,Interest_Area
0,CMP_U,Uber,Healthcare,Medium Enterprise,Franchise,Series A,USA,Marketing
1,CMP_W,Walmart,Healthcare,Small business,Public Sector,Bootstrapped,USA,Marketing
2,CMP_FMC,Ford Motor Co,Technology,Start-up,Public Sector,Series B,USA,Cloud
3,CMP_A,Amazon,Healthcare,Start-up,Public Sector,IPO,USA,Data
4,CMP_M,Microsoft,Retail,Start-up,Joint Venture,Seed,USA,Marketing


,Company_ID,Company_Name,Company_Industry,Company_Size,Company_Ownership,Company_Funding,Country,Interest_Area
1958,CMP_I_1996,InnovativeConsulting,Finance,Medium Enterprise,Franchise,IPO,Canada,Data
1959,CMP_G_1997,GlobalConsulting,Healthcare,Medium Enterprise,Private Sector,Series B,Canada,Cloud
1960,CMP_L_1998,LeadingPartners,Finance,Start-up,Joint Venture,IPO,Canada,Cloud
1961,CMP_I_1999,InnovativeTech,Healthcare,Small business,Public Sector,IPO,Canada,Marketing
1962,CMP_G_2000,GlobalIndustries,Finance,Small business,Franchise,Bootstrapped,Canada,Cloud


Generated 0 new companies. Total companies: 1963


In [32]:
#REGION
import pandas as pd
import numpy as np
import random
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# ---------- Google Sheets Authentication and Connection ----------
try:
    google_sheets_client = client
except NameError:
    print("Google Sheets client not found. Attempting to authenticate and create a new client.")
    try:
        scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
        creds = ServiceAccountCredentials.from_json_keyfile_name("/content/drive/MyDrive/CAPSTONE_FINAL/creds.json", scope)
        google_sheets_client = gspread.authorize(creds)
        print("Successfully authenticated and created Google Sheets client.")
    except FileNotFoundError:
        print("Error: 'creds.json' not found. Please ensure your service account credentials file is in the correct path.")
        raise
    except Exception as e:
        print(f"An error occurred during Google Sheets authentication: {e}")
        raise

try:
    spreadsheet
except NameError:
    print("Spreadsheet object not found. Attempting to open spreadsheet.")
    try:
        spreadsheet = google_sheets_client.open("CRM_Capstone") # Replace with your spreadsheet name
        print("Successfully opened spreadsheet.")
    except Exception as e:
        print(f"An error occurred while opening the spreadsheet: {e}")
        raise

# 1. Read existing Regions data from Google Sheets
try:
    worksheet_regions = spreadsheet.worksheet("Regions")
    df_existing_regions = pd.DataFrame(worksheet_regions.get_all_records())
    print(f"Successfully loaded {len(df_existing_regions)} existing regions from Google Sheets.")
except gspread.WorksheetNotFound:
    print("Regions sheet not found. Starting with an empty DataFrame for existing regions.")
    df_existing_regions = pd.DataFrame(columns=[
        "Region_ID", "Region_Name", "Region_Head", "Timezone", "Region_Target"
    ])
except Exception as e:
    print(f"An error occurred while loading existing regions: {e}")
    df_existing_regions = pd.DataFrame(columns=[
        "Region_ID", "Region_Name", "Region_Head", "Timezone", "Region_Target"
    ])


# Use unique Region_IDs from the combined df_lead
region_ids = df_lead['Region_ID'].unique().tolist()
timezones = ["IST", "EST", "PST", "CET", "GMT"]
countries_in_leads = df_lead['Country'].unique().tolist()

region_data = []
added_region_ids = set(df_existing_regions['Region_ID'].tolist()) # Get existing Region IDs

# Define value mappings for company size for target calculation
company_size_value = {
    "Start-up": 1,
    "Small business": 5,
    "Medium Enterprise": 10,
    "Large Enterprise": 20
}

# Use the combined df_company for calculating region targets
try:
    df_company
except NameError:
    print("df_company not found. Cannot calculate Region Targets based on companies.")
    # Create a dummy df_company to avoid errors, but targets will be random or 0
    df_company = pd.DataFrame(columns=['Company_ID', 'Country', 'Company_Size', 'Company_Ownership', 'Company_Funding', 'Interest_Area'])


for region_id in region_ids:
    if region_id not in added_region_ids:
        # Select a country that belongs to this region based on region_mapping and is in leads
        country_options = [c for r, data in region_mapping.items() if r == region_id for c in data['countries'] if c in countries_in_leads]
        country = random.choice(country_options) if country_options else "Unknown" # Handle case with no countries in leads for this region

        # Find reps assigned to this region (using the combined df_sales_rep if available, otherwise rep_df)
        try:
            df_sales_rep
            reps_in_region = df_sales_rep[df_sales_rep['Region_ID'] == region_id]['Rep_Name'].tolist()
        except NameError:
            # Fallback to initial rep_df if df_sales_rep is not defined
            reps_in_region = rep_df[rep_df['region'] == region_id]['rep_name'].tolist()


        region_head = random.choice(reps_in_region) if reps_in_region else "Unassigned"

        # Calculate Region_Target based on companies in the region using the combined df_company
        companies_in_region = df_company[df_company['Country'].apply(lambda x: x in region_mapping.get(region_id, {}).get('countries', []))]

        if not companies_in_region.empty:
             company_scores = companies_in_region.apply(
                lambda row: company_size_value.get(row['Company_Size'], 0) *
                            (2 if row['Company_Ownership'] == 'Private Sector' else 1) *
                            (2 if row['Company_Funding'] in ['Series B', 'IPO'] else 1) *
                            (2 if row['Interest_Area'] in region_mapping.get(region_id, {}).get('services', []) else 1),
                axis=1
            )
             raw_region_target = company_scores.sum()
             region_target = round(raw_region_target * 500, 2) # Adjusted scaling factor
        else:
             # Assign a default or random target if no companies are found for the region
             region_target = round(random.uniform(10000, 500000), 2) # Default random target


        region_data.append({
            "Region_ID": region_id,
            "Region_Name": f"Region_{region_id}",
            "Region_Head": region_head,
            "Timezone": random.choice(timezones),
            "Region_Target": region_target
        })
        added_region_ids.add(region_id)

# Combine existing and new regions
df_new_regions = pd.DataFrame(region_data)
df_region = pd.concat([df_existing_regions, df_new_regions], ignore_index=True)

display(df_region.head())
display(df_region.tail())
print(f"Generated {len(df_new_regions)} new regions. Total regions: {len(df_region)}")

Successfully loaded 6 existing regions from Google Sheets.


,Region_ID,Region_Name,Region_Head,Timezone,Region_Target
0,NA,Region_NA,Bob Johnson,PST,29878000
1,EU,Region_EU,Fiona Green,IST,229000
2,APAC,Region_APAC,Ian Gray,GMT,301000
3,ME,Region_ME,Oscar Clark,CET,85000
4,LA,Region_LA,Paula Young,CET,111000


,Region_ID,Region_Name,Region_Head,Timezone,Region_Target
1,EU,Region_EU,Fiona Green,IST,229000
2,APAC,Region_APAC,Ian Gray,GMT,301000
3,ME,Region_ME,Oscar Clark,CET,85000
4,LA,Region_LA,Paula Young,CET,111000
5,AF,Region_AF,Steve Wright,PST,186000


Generated 0 new regions. Total regions: 6


In [33]:
#SALES_REP
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# ---------- Google Sheets Authentication and Connection ----------
try:
    google_sheets_client = client
except NameError:
    print("Google Sheets client not found. Attempting to authenticate and create a new client.")
    try:
        scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
        creds = ServiceAccountCredentials.from_json_keyfile_name("/content/drive/MyDrive/CAPSTONE_FINAL/creds.json", scope)
        google_sheets_client = gspread.authorize(creds)
        print("Successfully authenticated and created Google Sheets client.")
    except FileNotFoundError:
        print("Error: 'creds.json' not found. Please ensure your service account credentials file is in the correct path.")
        raise
    except Exception as e:
        print(f"An error occurred during Google Sheets authentication: {e}")
        raise

try:
    spreadsheet
except NameError:
    print("Spreadsheet object not found. Attempting to open spreadsheet.")
    try:
        spreadsheet = google_sheets_client.open("CRM_Capstone") # Replace with your spreadsheet name
        print("Successfully opened spreadsheet.")
    except Exception as e:
        print(f"An error occurred while opening the spreadsheet: {e}")
        raise

# 1. Read existing Sales_Reps data from Google Sheets
try:
    worksheet_sales_reps = spreadsheet.worksheet("Sales_Reps")
    df_existing_sales_reps = pd.DataFrame(worksheet_sales_reps.get_all_records())
    print(f"Successfully loaded {len(df_existing_sales_reps)} existing sales reps from Google Sheets.")
except gspread.WorksheetNotFound:
    print("Sales_Reps sheet not found. Starting with an empty DataFrame for existing sales reps.")
    df_existing_sales_reps = pd.DataFrame(columns=[
        "Rep_ID", "Rep_Name", "Email", "Phone", "Region_ID", "Join_Date",
        "Current_Status", "Performance_Rating", "Total_Deals_Closed",
        "Avg_conversion_time", "Win_rate"
    ])
except Exception as e:
    print(f"An error occurred while loading existing sales reps: {e}")
    df_existing_sales_reps = pd.DataFrame(columns=[
        "Rep_ID", "Rep_Name", "Email", "Phone", "Region_ID", "Join_Date",
        "Current_Status", "Performance_Rating", "Total_Deals_Closed",
        "Avg_conversion_time", "Win_rate"
    ])


# Use unique Rep_IDs and their corresponding regions from the combined df_lead
# Also use the initial rep_df to get rep names and region distribution
try:
    rep_df
except NameError:
     # If rep_df is not defined, regenerate it
     print("rep_df not found. Regenerating Sales Reps for new rep generation.")
     rep_names = [
        "Alice Smith", "Bob Johnson", "Charlie Lee", "Diana King", "Edward Brown",
        "Fiona Green", "George White", "Hannah Black", "Ian Gray", "Jill Lewis",
        "Kevin Martin", "Laura Scott", "Mike Turner", "Nina Adams", "Oscar Clark",
        "Paula Young", "Quinn Hall", "Rachel Allen", "Steve Wright", "Tina Baker"
    ]
     region_rep_distribution = {"NA": 4, "EU": 4, "APAC": 5, "ME": 2, "LA": 3, "AF": 2}
     rep_list = []
     rep_name_index = 0
     for region, count in region_rep_distribution.items():
        for i in range(count):
            full_name = rep_names[rep_name_index]
            initials = "".join([part[0] for part in full_name.split()]).upper()
            rep_id = f"{initials}_{region}"
            rep_list.append({"rep_id": rep_id, "rep_name": full_name, "region": region})
            rep_name_index += 1
     rep_df = pd.DataFrame(rep_list)
     print("Sales Reps definition regenerated.")


sales_rep_data = []
added_rep_ids = set(df_existing_sales_reps['Rep_ID'].tolist()) # Get existing Rep IDs

# Add reps from rep_df who are not in the existing data
for index, rep_row in rep_df.iterrows():
    rep_id = rep_row['rep_id']
    if rep_id not in added_rep_ids:
        region_id = rep_row['region']
        rep_name = rep_row['rep_name']

        sales_rep_data.append({
            "Rep_ID": rep_id,
            "Rep_Name": rep_name,
            "Email": f"{rep_id.lower()}@crmcorp.com",
            "Phone": f"+91-9{random.randint(100000000,999999999)}",
            "Region_ID": region_id,
            "Join_Date": (datetime(2020, 1, 1) + timedelta(days=random.randint(0, 1000))).date(), # Store as date only
            "Current_Status": random.choice(["Active", "Inactive", "On Leave"]),
            "Performance_Rating": round(random.uniform(1.0, 5.0), 1),
            "Total_Deals_Closed": random.randint(0, 500),
            "Avg_conversion_time": round(random.uniform(3.0, 30.0), 2),
            "Win_rate": round(random.uniform(0.1, 0.9), 2)
        })
        added_rep_ids.add(rep_id)

# Combine existing and new sales reps
df_new_sales_reps = pd.DataFrame(sales_rep_data)
df_sales_rep = pd.concat([df_existing_sales_reps, df_new_sales_reps], ignore_index=True)

display(df_sales_rep.head())
display(df_sales_rep.tail())
print(f"Generated {len(df_new_sales_reps)} new sales reps. Total sales reps: {len(df_sales_rep)}")

Successfully loaded 20 existing sales reps from Google Sheets.


,Rep_ID,Rep_Name,Email,Phone,Region_ID,Join_Date,Current_Status,Performance_Rating,Total_Deals_Closed,Avg_conversion_time,Win_rate
0,BJ_NA,Bob Johnson,bj_na@crmcorp.com,-9490641556,NA,2021-03-10,On Leave,1.4,110,14.38,0.34
1,DK_NA,Diana King,dk_na@crmcorp.com,-9657433913,NA,2021-02-05,Active,1.6,390,22.91,0.34
2,CL_NA,Charlie Lee,cl_na@crmcorp.com,-9441914786,NA,2020-02-05,On Leave,1.0,65,11.93,0.13
3,AS_NA,Alice Smith,as_na@crmcorp.com,-9106003361,NA,2022-09-10,On Leave,2.5,3,13.89,0.89
4,EB_EU,Edward Brown,eb_eu@crmcorp.com,-9768974832,EU,2020-04-05,Active,2.2,357,27.81,0.66


,Rep_ID,Rep_Name,Email,Phone,Region_ID,Join_Date,Current_Status,Performance_Rating,Total_Deals_Closed,Avg_conversion_time,Win_rate
15,RA_LA,Rachel Allen,ra_la@crmcorp.com,-9443961384,LA,2022-04-14,Active,2.6,318,6.26,0.37
16,QH_LA,Quinn Hall,qh_la@crmcorp.com,-9271909851,LA,2020-07-08,Active,1.3,51,14.21,0.19
17,SW_AF,Steve Wright,sw_af@crmcorp.com,-9648766672,AF,2021-01-13,On Leave,2.8,303,11.99,0.87
18,TB_AF,Tina Baker,tb_af@crmcorp.com,-9645421906,AF,2020-04-22,Inactive,2.6,263,16.66,0.67
19,KM_APAC,Kevin Martin,km_apac@crmcorp.com,-9692745290,APAC,2021-12-09,Active,5.0,79,24.24,0.18


Generated 0 new sales reps. Total sales reps: 20


In [35]:
#STAGE HISTORY
import pandas as pd
import random
from datetime import datetime, timedelta
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# ---------- Google Sheets Authentication and Connection ----------
try:
    google_sheets_client = client
except NameError:
    print("Google Sheets client not found. Attempting to authenticate and create a new client.")
    try:
        scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
        creds = ServiceAccountCredentials.from_json_keyfile_name("/content/drive/MyDrive/CAPSTONE_FINAL/creds.json", scope)
        google_sheets_client = gspread.authorize(creds)
        print("Successfully authenticated and created Google Sheets client.")
    except FileNotFoundError:
        print("Error: 'creds.json' not found. Please ensure your service account credentials file is in the correct path.")
        raise
    except Exception as e:
        print(f"An error occurred during Google Sheets authentication: {e}")
        raise

try:
    spreadsheet
except NameError:
    print("Spreadsheet object not found. Attempting to open spreadsheet.")
    try:
        spreadsheet = google_sheets_client.open("CRM_Capstone") # Replace with your spreadsheet name
        print("Successfully opened spreadsheet.")
    except Exception as e:
        print(f"An error occurred while opening the spreadsheet: {e}")
        raise

# 1. Read existing Stage_History data from Google Sheets
try:
    worksheet_stage_history = spreadsheet.worksheet("Stage_History")
    df_existing_stage_history = pd.DataFrame(worksheet_stage_history.get_all_records())
    # Ensure Timestamp is in datetime format for comparison
    df_existing_stage_history['Timestamp'] = pd.to_datetime(df_existing_stage_history['Timestamp'], errors='coerce')
    print(f"Successfully loaded {len(df_existing_stage_history)} existing stage history entries from Google Sheets.")
except gspread.WorksheetNotFound:
    print("Stage_History sheet not found. Starting with an empty DataFrame for existing stage history.")
    df_existing_stage_history = pd.DataFrame(columns=[
        "History_ID", "Lead_ID", "Stage", "Timestamp", "Rep_ID", "Stage_Sequence", "Notes", "Stage_Engagement (in Days)"
    ])
    df_existing_stage_history['Timestamp'] = pd.to_datetime(df_existing_stage_history['Timestamp']) # Ensure Timestamp column exists and is datetime
except Exception as e:
    print(f"An error occurred while loading existing stage history: {e}")
    df_existing_stage_history = pd.DataFrame(columns=[
        "History_ID", "Lead_ID", "Stage", "Timestamp", "Rep_ID", "Stage_Sequence", "Notes", "Stage_Engagement (in Days)"
    ])
    df_existing_stage_history['Timestamp'] = pd.to_datetime(df_existing_stage_history['Timestamp'])


# Define stages in the desired funnel order based on user provided ranges
funnel_stages = [
    "Lead Captured",
    "Qualified",
    "Contacted",
    "Responded",
    "Demo Scheduled",
    "Demo",
    "Demo Completed",
    "Proposal Sent",
    "Proposal",
    "Negotiation"
]

stage_history_data = []

# Start history_id counter from the maximum existing ID + 1
if not df_existing_stage_history.empty:
    history_id = df_existing_stage_history['History_ID'].astype(int).max() + 1
else:
    history_id = 1


# Define realistic time ranges for progression between stages (in days)
# Using the specific ranges provided by the user
stage_time_ranges = {
    "Lead Captured": (2, 7),        # 2–7 days to qualification
    "Qualified": (4, 9),            # 4–9 days to contact
    "Contacted": (5, 12),           # 5–12 days to responded/demo
    "Responded": (3, 6),            # 3–6 days to demo scheduled
    "Demo Scheduled": (4, 8),       # 4–8 days to demo
    "Demo": (2, 3),                 # 2–3 days to complete demo
    "Demo Completed": (4, 9),       # 4–9 days to proposal
    "Proposal Sent": (5, 11),       # 5–11 days to proposal
    "Proposal": (6, 13),            # 6–13 days to negotiation
    "Negotiation": (7, 15)          # 7–15 days to close
}

def add_random_time_of_day(date):
    """Adds a random time of day (hour, minute, second) to a date."""
    # Ensure 'date' is a datetime object before adding timedelta
    if isinstance(date, datetime):
        return date + timedelta(hours=random.randint(0, 23), minutes=random.randint(0, 59), seconds=random.randint(0, 59))
    elif isinstance(date, pd.Timestamp):
         return date + timedelta(hours=random.randint(0, 23), minutes=random.randint(0, 59), seconds=random.randint(0, 59))
    else:
        # If it's a date object, convert to datetime before adding time
        return datetime(date.year, date.month, date.day) + timedelta(hours=random.randint(0, 23), minutes=random.randint(0, 59), seconds=random.randint(0, 59))


# Process leads from the combined df_lead
for index, lead in df_lead.iterrows():
    lead_id = lead.Lead_ID
    final_stage = lead['Current_Stage']

    # Determine the sequence of stages the lead went through, stopping at the Current_Stage
    # Handle 'Ongoing' stages by extracting the base stage name
    base_final_stage = final_stage.replace("Ongoing - ", "")

    try:
        if base_final_stage in ["Deal Won", "Deal Lost"]:
             # Find the index of the final stage in the funnel_stages.
             # If the final stage is not in funnel_stages (i.e., Won/Lost),
             # the history goes up to Negotiation (the last stage before Won/Lost).
             try:
                 final_stage_index_in_funnel = funnel_stages.index(base_final_stage)
                 # If the final stage is in funnel_stages, history goes up to that stage.
                 ordered_stages = funnel_stages[:final_stage_index_in_funnel + 1]
             except ValueError:
                 # If final stage is 'Deal Won' or 'Deal Lost', history goes up to 'Negotiation'
                 if "Negotiation" in funnel_stages:
                     final_history_stage_index = funnel_stages.index("Negotiation")
                     ordered_stages_for_history = funnel_stages[:final_history_stage_index + 1]
                 else:
                     # Fallback if Negotiation is not in funnel_stages
                     ordered_stages_for_history = funnel_stages # Include all stages if structure is different

                 # Add the actual final stage ('Deal Won' or 'Deal Lost') as the last entry in history
                 ordered_stages = ordered_stages_for_history + [base_final_stage]

        elif base_final_stage in funnel_stages:
            # If the final stage is an intermediate stage, history goes up to that stage
            final_stage_index = funnel_stages.index(base_final_stage)
            ordered_stages = funnel_stages[:final_stage_index + 1]
        else:
             # If the assigned stage is not in funnel_stages or is invalid, default to "Lead Captured"
             print(f"Warning: Lead {lead.Lead_ID} has an invalid final stage '{final_stage}'. Setting history to 'Lead Captured'.")
             ordered_stages = ["Lead Captured"]
             base_final_stage = "Lead Captured" # Update base_final_stage variable as well


    except ValueError:
        # If the assigned stage is not in funnel_stages or is invalid, default to "Lead Captured"
        print(f"Warning: Lead {lead.Lead_ID} has an invalid final stage '{final_stage}'. Setting history to 'Lead Captured'.")
        ordered_stages = ["Lead Captured"]
        base_final_stage = "Lead Captured" # Update base_final_stage variable as well


    start_date_str = lead.Created_Date
    last_updated_date_str = lead.Last_Updated

    # Convert to datetime objects
    try:
        start_date = datetime.strptime(start_date_str, "%Y-%m-%d %H:%M:%S")
    except (ValueError, TypeError):
         print(f"Warning: Could not parse Created_Date '{start_date_str}' for Lead {lead_id}. Using current time.")
         start_date = datetime.now()

    try:
        last_updated_date = datetime.strptime(last_updated_date_str, "%Y-%m-%d %H:%M:%S")
    except (ValueError, TypeError):
         print(f"Warning: Could not parse Last_Updated '{last_updated_date_str}' for Lead {lead_id}. Using Created_Date + 1 day.")
         last_updated_date = start_date + timedelta(days=1)

    # Ensure last_updated is not before start_date
    if last_updated_date < start_date:
        last_updated_date = start_date + timedelta(seconds=random.randint(1, 60))


    # Get existing history entries for this lead from the loaded data
    existing_history_for_lead = df_existing_stage_history[df_existing_stage_history['Lead_ID'] == lead_id].copy()

    # Track generated history entries for this lead to avoid generating duplicates for stages already present in existing history
    generated_stages_for_lead = set(existing_history_for_lead['Stage'].tolist())


    current_timestamp = start_date
    for seq, stage in enumerate(ordered_stages, start=1):
        # Only generate history for stages that are not already in the existing history for this lead
        if stage not in generated_stages_for_lead:
            timestamp = current_timestamp

            if seq == 1:
                timestamp = add_random_time_of_day(start_date)
                if timestamp < start_date:
                    timestamp = start_date + timedelta(seconds=random.randint(1, 60))
            elif seq == len(ordered_stages):
                timestamp = last_updated_date
                if timestamp <= current_timestamp:
                     timestamp = current_timestamp + timedelta(seconds=random.randint(1, 60))
            else:
                 remaining_duration_seconds = (last_updated_date - current_timestamp).total_seconds()
                 remaining_transitions = len(ordered_stages) - seq

                 if remaining_transitions > 0:
                     min_days, max_days = stage_time_ranges.get(stage, (1, 1))
                     min_seconds = min_days * 24 * 3600
                     max_seconds = max_days * 24 * 3600

                     # Calculate an estimated minimum time for remaining stages to avoid generating timestamps that are too close to the end date
                     remaining_stages_in_path = ordered_stages[seq:] # Include current stage for range lookup, but consider remaining path length
                     estimated_min_remaining_days = sum(stage_time_ranges.get(s, (1, 1))[0] for s in remaining_stages_in_path)

                     min_seconds_for_remaining_stages = estimated_min_remaining_days * 24 * 3600

                     max_possible_increment_seconds = remaining_duration_seconds - min_seconds_for_remaining_stages
                     max_possible_increment_seconds = max(1, max_possible_increment_seconds) if remaining_transitions > 0 else 0


                     lower_bound_seconds = max(min_seconds, 1)
                     upper_bound_seconds = min(max_seconds, max_possible_increment_seconds, remaining_duration_seconds)
                     upper_bound_seconds = max(lower_bound_seconds, upper_bound_seconds) # Ensure upper is at least lower


                     # Handle case where there is no time left for the next stage or remaining stages
                     if upper_bound_seconds <= lower_bound_seconds and remaining_transitions > 0:
                          # If no time is left, assign the timestamp very close to the last updated date
                          timestamp = last_updated_date - timedelta(seconds=random.randint(1, 60)) # Place just before end
                          # Ensure it's still after the current timestamp
                          if timestamp <= current_timestamp:
                               timestamp = current_timestamp + timedelta(seconds=random.randint(1, 60))

                     else:
                          time_to_add_seconds = random.uniform(lower_bound_seconds, upper_bound_seconds)
                          next_timestamp = current_timestamp + timedelta(seconds=time_to_add_seconds)

                          # Small increment if exact match or regression (should be handled by range logic, but safeguard)
                          while next_timestamp <= current_timestamp:
                               next_timestamp = current_timestamp + timedelta(seconds=random.uniform(1, 60))

                          timestamp = next_timestamp
                 else:
                     # If it's the second to last stage and remaining_transitions is 0, the last stage timestamp will be used next
                     # Just set the timestamp to be slightly before the last updated date if it hasn't been reached yet
                     if current_timestamp < last_updated_date:
                          timestamp = last_updated_date - timedelta(seconds=random.randint(1, 60))
                          if timestamp <= current_timestamp:
                               timestamp = current_timestamp + timedelta(seconds=random.randint(1, 60))
                     else:
                          timestamp = current_timestamp + timedelta(seconds=random.randint(1, 60)) # Small increment


            # Add the new history entry
            stage_history_data.append({
                "History_ID": history_id, # Assign a unique ID for each new entry
                "Lead_ID": lead_id,
                "Stage": stage,
                "Timestamp": timestamp,
                "Rep_ID": lead.Rep_ID, # Use Rep_ID from df_lead
                "Stage_Sequence": seq,
                "Notes": f"Notes for stage {stage} for Lead {lead_id}"
            })
            history_id += 1
            current_timestamp = timestamp
            generated_stages_for_lead.add(stage) # Add the generated stage to the set

        else:
            # If the stage already exists in the loaded history for this lead,
            # update current_timestamp to the latest timestamp of this stage from the existing history
            latest_existing_timestamp_for_stage = existing_history_for_lead[
                existing_history_for_lead['Stage'] == stage
            ]['Timestamp'].max()
            if pd.notna(latest_existing_timestamp_for_stage):
                current_timestamp = latest_existing_timestamp_for_stage


# Combine existing and new stage history entries
print(f"Generated {len(stage_history_data)} new raw stage history entries.") # Debug print
df_new_stage_history = pd.DataFrame(stage_history_data)

# Convert Timestamp to datetime, coercing errors
if 'Timestamp' in df_new_stage_history.columns:
    df_new_stage_history['Timestamp'] = pd.to_datetime(df_new_stage_history['Timestamp'], errors='coerce')

    # Drop any rows with invalid timestamps created by coercion
    initial_new_rows = len(df_new_stage_history)
    df_new_stage_history.dropna(subset=['Timestamp'], inplace=True)
    if len(df_new_stage_history) < initial_new_rows:
        print(f"Warning: Dropped {initial_new_rows - len(df_new_stage_history)} new stage history entries due to invalid timestamps.")
else:
    print("Warning: 'Timestamp' column not found in df_new_stage_history. Cannot convert or drop NaT values.")


# Combine existing and new data
# When concatenating, ensure columns match and handle potential duplicates based on Lead_ID, Stage, and Timestamp
# We will rely on the ETL cell's deduplication for robustness, but a simple concat here
# allows us to see the potential new entries before final deduplication.
df_stage_history = pd.concat([df_existing_stage_history, df_new_stage_history], ignore_index=True)

# Sort to ensure correct calculation of Stage_Engagement (this will be recalculated in ETL for final dataset)
df_stage_history = df_stage_history.sort_values(by=['Lead_ID', 'Timestamp']).reset_index(drop=True)

# Recalculate Stage_Engagement (in Days) for the combined DataFrame
# This calculation will also be done in the ETL cell on the final, deduplicated data.
# Doing it here provides an intermediate view but might not be the final correct value
# if deduplication in ETL removes entries.
stage_engagement_days = []

for lead_id, group in df_stage_history.groupby('Lead_ID'):
    group = group.sort_values(by='Timestamp').reset_index(drop=True)
    for i in range(len(group)):
        if i == 0:
            days = 0
        else:
            time_diff = group.iloc[i]['Timestamp'] - group.iloc[i-1]['Timestamp']
            days = round(time_diff.total_seconds() / (24 * 3600))
            days = max(days, 0)
        stage_engagement_days.append(days)

# Assign the calculated engagement days
if len(stage_engagement_days) == len(df_stage_history):
    df_stage_history['Stage_Engagement (in Days)'] = stage_engagement_days
else:
    print(f"Warning: Length mismatch in stage engagement calculation ({len(stage_engagement_days)} vs {len(df_stage_history)}). Skipping assignment in generation cell.")
    # As a fallback, fill with a default or recalculate in a simpler way
    # For this case, let's keep the warning and rely on the ETL cell's recalculation


# Display sample
display(df_lead.head())
display(df_lead.tail())
print(f"Total leads (existing + new): {len(df_lead)}") # Display total leads count

display(df_stage_history.head())
display(df_stage_history.tail())
print(f"Generated {len(df_new_stage_history)} new stage history entries. Total stage history entries: {len(df_stage_history)}")

Successfully loaded 37759 existing stage history entries from Google Sheets.
Generated 0 new raw stage history entries.


,Lead_ID,Lead_Name,Company_Name,Source,Region_ID,Rep_ID,Created_Date,Current_Stage,Email,Phone,...,Deal_Source,Company_ID,Engagement_Level,Follow_Up_Threshold,Project_Term,Budget_Tier,Country,Company_Industry,Service_Category,Deal_Status_Category
0,LDUBR-NA-001,Uber Lead,Uber,Instagram Ads,NA,BJ_NA,2024-10-28 4:03:50,Deal Lost,contact_70@uber.com,-528015363,...,Outbound,CMP_U,Medium,7,>24 months,100L+,USA,Technology,Marketing,Deal Lost
1,LDWMT-NA-002,Walmart Lead,Walmart,Google,NA,DK_NA,2024-10-17 12:47:49,Deal Won,contact_61@walmart.com,-587713714,...,Inbound,CMP_W,High,5,6-12 months,50-100L,USA,Retail,Cloud,Deal Won
2,LDFRD-NA-003,Ford Motor Co Lead,Ford Motor Co,Google,NA,DK_NA,2024-04-20 15:52:48,Deal Won,contact_39@fordmotorco.com,-4240414113,...,Inbound,CMP_FMC,Low,7,0-6 months,20-50L,USA,Retail,Data,Deal Won
3,LDAMZ-NA-004,Amazon Lead,Amazon,LinkedIn Ads,NA,CL_NA,2025-02-23 12:08:37,Deal Won,contact_65@amazon.com,-295989703,...,Outbound,CMP_A,Medium,5,0-6 months,100L+,USA,Finance,Marketing,Deal Won
4,LDMST-NA-005,Microsoft Lead,Microsoft,Instagram Ads,NA,DK_NA,2024-05-30 15:44:04,Deal Lost,contact_77@microsoft.com,-6876180902,...,Outbound,CMP_M,Low,7,6-12 months,20-50L,USA,Healthcare,Cloud,Deal Lost


,Lead_ID,Lead_Name,Company_Name,Source,Region_ID,Rep_ID,Created_Date,Current_Stage,Email,Phone,...,Deal_Source,Company_ID,Engagement_Level,Follow_Up_Threshold,Project_Term,Budget_Tier,Country,Company_Industry,Service_Category,Deal_Status_Category
1998,LDINNO-NA-1999,InnovativeTech Lead,InnovativeTech,Cold Calling,NA,AS_NA,2024-12-08 8:12:20,Deal Won,contact_22@innovativetech.com,-3240368818,...,Outbound,CMP_I_1999,Medium,3,>24 months,20-50L,Canada,Healthcare,Marketing,Deal Won
1999,LDGLOB-NA-2000,GlobalIndustries Lead,GlobalIndustries,Referrals from Existing Customers,NA,DK_NA,2024-09-16 9:50:44,Deal Won,contact_89@globalindustries.com,-9218476218,...,Inbound,CMP_G_2000,High,7,>24 months,50-100L,Canada,Finance,Data,Deal Won
2000,LDINNO-NA-1997,GlobalConsulting Lead,GlobalConsulting,Google,NA,BJ_NA,2024-03-08 1:21:37,Deal Won,contact_81@globalconsulting.com,-3315598761,...,Inbound,CMP_G_1997,Medium,7,6-12 months,100L+,Canada,Retail,Marketing,Deal Won
2001,LDINNO-NA-1998,LeadingPartners Lead,LeadingPartners,LinkedIn,NA,BJ_NA,2024-02-20 17:13:52,Deal Lost,contact_11@leadingpartners.com,-1799890194,...,Inbound,CMP_L_1998,Medium,3,>24 months,50-100L,Canada,Retail,Marketing,Deal Lost
2002,LDINNO-NA-2000,GlobalIndustries Lead,GlobalIndustries,Referrals from Existing Customers,NA,DK_NA,2024-09-16 9:50:44,Deal Won,contact_89@globalindustries.com,-9218476218,...,Inbound,CMP_G_2000,High,7,>24 months,50-100L,Canada,Finance,Data,Deal Won


Total leads (existing + new): 2003


,History_ID,Lead_ID,Stage,Timestamp,Rep_ID,Stage_Sequence,Notes,Stage_Engagement (in Days)
0,334,LDABB-EU-032,Lead Captured,2023-09-09 00:06:43,GW_EU,1,Notes for stage Lead Captured for Lead LDABB-E...,0
1,19608,LDABB-EU-032,Lead Captured,2023-09-09 09:16:35,GW_EU,1,Notes for stage Lead Captured for Lead LDABB-E...,0
2,335,LDABB-EU-032,Qualified,2023-09-13 00:06:43,GW_EU,2,Notes for stage Qualified for Lead LDABB-EU-032,4
3,19609,LDABB-EU-032,Qualified,2023-09-13 09:16:35,GW_EU,2,Notes for stage Qualified for Lead LDABB-EU-032,0
4,336,LDABB-EU-032,Contacted,2023-09-18 00:06:43,GW_EU,3,Notes for stage Contacted for Lead LDABB-EU-032,5


,History_ID,Lead_ID,Stage,Timestamp,Rep_ID,Stage_Sequence,Notes,Stage_Engagement (in Days)
37754,901,LDZNR-APAC-087,Proposal,2024-08-04 09:34:26,JL_APAC,9,Notes for stage Proposal for Lead LDZNR-APAC-087,0
37755,20176,LDZNR-APAC-087,Negotiation,2024-08-11 09:19:08,JL_APAC,10,Notes for stage Negotiation for Lead LDZNR-APA...,7
37756,20177,LDZNR-APAC-087,Deal Won,2024-08-11 09:19:35,JL_APAC,11,Notes for stage Deal Won for Lead LDZNR-APAC-087,0
37757,902,LDZNR-APAC-087,Negotiation,2024-08-11 09:34:26,JL_APAC,10,Notes for stage Negotiation for Lead LDZNR-APA...,0
37758,903,LDZNR-APAC-087,Deal Won,2024-08-11 09:34:45,JL_APAC,11,Notes for stage Deal Won for Lead LDZNR-APAC-087,0


Generated 0 new stage history entries. Total stage history entries: 37759


In [36]:
# DEALS GENERATION
import pandas as pd
import random
from datetime import datetime, timedelta
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# ---------- Google Sheets Authentication and Connection ----------
try:
    google_sheets_client = client
except NameError:
    print("Google Sheets client not found. Attempting to authenticate and create a new client.")
    try:
        scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
        creds = ServiceAccountCredentials.from_json_keyfile_name("/content/drive/MyDrive/CAPSTONE_FINAL/creds.json", scope)
        google_sheets_client = gspread.authorize(creds)
        print("Successfully authenticated and created Google Sheets client.")
    except FileNotFoundError:
        print("Error: 'creds.json' not found. Please ensure your service account credentials file is in the correct path.")
        raise
    except Exception as e:
        print(f"An error occurred during Google Sheets authentication: {e}")
        raise

try:
    spreadsheet
except NameError:
    print("Spreadsheet object not found. Attempting to open spreadsheet.")
    try:
        spreadsheet = google_sheets_client.open("CRM_Capstone") # Replace with your spreadsheet name
        print("Successfully opened spreadsheet.")
    except Exception as e:
        print(f"An error occurred while opening the spreadsheet: {e}")
        raise

# 1. Read existing Deals data from Google Sheets
try:
    worksheet_deals = spreadsheet.worksheet("Deals")
    df_existing_deals = pd.DataFrame(worksheet_deals.get_all_records())
    print(f"Successfully loaded {len(df_existing_deals)} existing deals from Google Sheets.")
except gspread.WorksheetNotFound:
    print("Deals sheet not found. Starting with an empty DataFrame for existing deals.")
    df_existing_deals = pd.DataFrame(columns=[
        "Deal_ID", "Lead_ID", "Company_ID", "Amount_Display", "Amount_Numeric",
        "Status", "Created_Date", "Closed_Date", "Rep_ID", "Total_Duration_Days"
    ])
except Exception as e:
    print(f"An error occurred while loading existing deals: {e}")
    df_existing_deals = pd.DataFrame(columns=[
        "Deal_ID", "Lead_ID", "Company_ID", "Amount_Display", "Amount_Numeric",
        "Status", "Created_Date", "Closed_Date", "Rep_ID", "Total_Duration_Days"
    ])


def generate_deals(df_lead, df_stage_history, df_existing_deals):
    new_deals_data = []
    # Start deal_id_counter from the maximum existing ID + 1
    if not df_existing_deals.empty:
        # Extract numeric part from Deal_ID (assuming format "DEAL_XXXX")
        df_existing_deals['Deal_Numeric_ID'] = df_existing_deals['Deal_ID'].str.replace('DEAL_', '').astype(int)
        deal_id_counter = df_existing_deals['Deal_Numeric_ID'].max() + 1
        df_existing_deals.drop(columns=['Deal_Numeric_ID'], inplace=True) # Drop the temporary column
    else:
        deal_id_counter = 1

    # Define amount ranges for each budget tier (in Lakhs)
    budget_tier_amounts = {
        "20-50L": (20, 50),
        "50-100L": (50, 100),
        "100L+": (100, 500)
    }

    # Process leads from the combined df_lead that should have a deal
    # This includes leads that reached 'Negotiation', 'Deal Won', 'Deal Lost', or are 'Ongoing'
    leads_with_deals = df_lead[
        df_lead['Current_Stage'].isin(['Negotiation', 'Deal Won', 'Deal Lost']) |
        df_lead['Current_Stage'].str.startswith('Ongoing - ')
    ].copy()

    # Get existing Lead IDs that already have deals
    existing_deal_lead_ids = df_existing_deals['Lead_ID'].tolist()

    for index, lead in leads_with_deals.iterrows():
        lead_id = lead['Lead_ID']

        # Only generate a new deal if this Lead_ID does not already have an existing deal
        if lead_id not in existing_deal_lead_ids:
            current_stage = lead['Current_Stage']
            deal_status_category = lead['Deal_Status_Category']

            # Find the 'Lead Captured' timestamp from the combined stage history
            lead_captured_ts_row = df_stage_history[
                (df_stage_history['Lead_ID'] == lead_id) & (df_stage_history['Stage'] == 'Lead Captured')
            ]
            lead_captured_ts = lead_captured_ts_row['Timestamp'].min() if not lead_captured_ts_row.empty else pd.NaT

            if pd.isna(lead_captured_ts):
                print(f"Warning: Skipping lead {lead_id} as it has no 'Lead Captured' stage history in the combined data.")
                continue


            budget_tier = lead['Budget_Tier']
            amount_range = budget_tier_amounts.get(budget_tier, (1, 10))
            amount_in_lakhs = round(random.uniform(amount_range[0], amount_range[1]), 2)

            if amount_in_lakhs >= 100:
                 amount_display = f"{amount_in_lakhs / 100:.2f} Crore"
                 amount_numeric = amount_in_lakhs * 100000
            else:
                 amount_display = f"{amount_in_lakhs:.2f} Lakhs"
                 amount_numeric = amount_in_lakhs * 100000

            closed_date = None
            total_duration_days = None

            if current_stage in ['Deal Won', 'Deal Lost']:
                # Find the timestamp of the 'Deal Won' or 'Deal Lost' stage
                closed_ts_row = df_stage_history[
                    (df_stage_history['Lead_ID'] == lead_id) & (df_stage_history['Stage'].isin(['Deal Won', 'Deal Lost']))
                ]
                closed_ts = closed_ts_row['Timestamp'].max() if not closed_ts_row.empty else pd.NaT

                if pd.notna(closed_ts) and pd.notna(lead_captured_ts):
                    closed_date = closed_ts.date()
                    total_duration_days = (closed_ts - lead_captured_ts).days
                elif pd.notna(lead_captured_ts):
                     # If closed_ts is not found but lead_captured_ts is, calculate duration to now
                     total_duration_days = (datetime.now() - lead_captured_ts).days


            elif current_stage.startswith('Ongoing - '):
                # For ongoing deals, calculate duration from Lead Captured Date to current date
                if pd.notna(lead_captured_ts):
                     total_duration_days = (datetime.now() - lead_captured_ts).days


            status = deal_status_category if deal_status_category in ['Ongoing - Active', 'Ongoing - Cold'] else current_stage


            new_deals_data.append({
                "Deal_ID": f"DEAL_{deal_id_counter:04}",
                "Lead_ID": lead_id,
                "Company_ID": lead['Company_ID'],
                "Amount_Display": amount_display,
                "Amount_Numeric": amount_numeric,
                "Status": status,
                "Created_Date": lead_captured_ts.date(), # Store as date only
                "Closed_Date": closed_date, # Store as date only
                "Rep_ID": lead['Rep_ID'],
                "Total_Duration_Days": total_duration_days
            })
            deal_id_counter += 1

    return pd.DataFrame(new_deals_data)

# Generate new deals based on the combined df_lead and df_stage_history
df_new_deals = generate_deals(df_lead, df_stage_history, df_existing_deals)

# Combine existing and new deals
df_deals = pd.concat([df_existing_deals, df_new_deals], ignore_index=True)


display(df_deals.head())
display(df_deals.tail())
print(f"Generated {len(df_new_deals)} new deals. Total deals: {len(df_deals)}")

Successfully loaded 2003 existing deals from Google Sheets.


,Deal_ID,Lead_ID,Company_ID,Amount_Display,Amount_Numeric,Status,Created_Date,Closed_Date,Rep_ID,Total_Duration_Days
0,DEAL_0001,LDUBR-NA-001,CMP_U,3.31 Crore,33062000,Deal Lost,2024-10-28,2024-12-20,BJ_NA,53
1,DEAL_0002,LDWMT-NA-002,CMP_W,72.97 Lakhs,7297000,Deal Won,2024-10-18,2024-11-27,DK_NA,40
2,DEAL_0003,LDFRD-NA-003,CMP_FMC,44.46 Lakhs,4446000,Deal Won,2024-04-21,2024-05-31,DK_NA,40
3,DEAL_0004,LDAMZ-NA-004,CMP_A,3.24 Crore,32449000,Deal Won,2025-02-24,2025-04-05,CL_NA,40
4,DEAL_0005,LDMST-NA-005,CMP_M,41.32 Lakhs,4132000,Deal Lost,2024-05-31,2024-07-18,DK_NA,48


,Deal_ID,Lead_ID,Company_ID,Amount_Display,Amount_Numeric,Status,Created_Date,Closed_Date,Rep_ID,Total_Duration_Days
1998,DEAL_1999,LDINNO-NA-1999,CMP_I_1999,44.07 Lakhs,4407000,Deal Won,2024-12-09,2025-01-31,AS_NA,53
1999,DEAL_2000,LDGLOB-NA-2000,CMP_G_2000,70.67 Lakhs,7067000,Deal Won,2024-09-16,2024-10-26,DK_NA,40
2000,DEAL_2001,LDINNO-NA-1997,CMP_G_1997,1.37 Crore,13734000,Deal Won,2024-03-08,2024-04-17,BJ_NA,40
2001,DEAL_2002,LDINNO-NA-1998,CMP_L_1998,84.83 Lakhs,8483000,Deal Lost,2024-02-21,2024-04-01,BJ_NA,40
2002,DEAL_2003,LDINNO-NA-2000,CMP_G_2000,55.61 Lakhs,5561000,Deal Won,2024-09-17,2024-10-27,DK_NA,40


Generated 0 new deals. Total deals: 2003


In [15]:
!pip install --upgrade gspread gspread_dataframe

import gspread
from gspread_dataframe import set_with_dataframe
from google.colab import auth
auth.authenticate_user()


In [16]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 3.4 MB/s eta 0:00:00


In [17]:
# Use the spreadsheet name you created
spreadsheet = client.open("CRM_Capstone")

# Sheet mappings
sheet_mapping = {
    "Leads": df_lead,
    "Sales_Reps": df_sales_rep,
    "Regions": df_region,
    "Stage_History": df_stage_history,
    "Deals":df_deals,
    "Companies": df_company
}

# Write to each sheet
for sheet_name, df in sheet_mapping.items():
    try:
        worksheet = spreadsheet.worksheet(sheet_name)
    except gspread.WorksheetNotFound:
        worksheet = spreadsheet.add_worksheet(title=sheet_name, rows=df.shape[0], cols=df.shape[1])

    worksheet.clear()
    set_with_dataframe(worksheet, df)

print("All DataFrames written to Google Sheets.")

All DataFrames written to Google Sheets.


In [37]:
# Full ETL pipeline (Google Sheet -> Clean -> Google Sheet -> SQLite + SQL Modeling + Google Drive CSV + BigQuery)

import pandas as pd
import re
import sqlite3
import gspread
from gspread_dataframe import set_with_dataframe
import urllib.parse # Import for URL encoding
from sqlalchemy import create_engine # Import for SQLAlchemy engine
from google.colab import auth # Import auth for Google Colab authentication
from oauth2client.service_account import ServiceAccountCredentials # Import for service account credentials
from google.colab import drive # Import drive for mounting Google Drive
from google.cloud import bigquery # Import BigQuery client
from datetime import datetime # Import datetime


# ---------- Google Sheets Authentication and Connection ----------
try:
    # Attempt to use the existing client object if defined
    # Renaming to google_sheets_client to avoid conflict with bigquery.Client
    google_sheets_client = client
except NameError:
    print("Google Sheets client not found. Attempting to authenticate and create a new client.")
    # Authenticate using service account credentials
    try:
        # Assuming the creds.json file path is defined or can be accessed
        # If your creds.json path is different, update this line
        scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
        creds = ServiceAccountCredentials.from_json_keyfile_name("/content/drive/MyDrive/CAPSTONE_FINAL/creds.json", scope)
        google_sheets_client = gspread.authorize(creds)
        print("Successfully authenticated and created Google Sheets client.")
    except FileNotFoundError:
        print("Error: 'creds.json' not found. Please ensure your service account credentials file is in the correct path.")
        # Fallback to Colab authentication if service account fails
        try:
            auth.authenticate_user()
            google_sheets_client = gspread.service_account_manager().authorize()
            print("Successfully authenticated using Google Colab user credentials.")
        except Exception as e:
            print(f"Error authenticating with Google Colab credentials: {e}")
            raise # Re-raise if both authentication methods fails
    except Exception as e:
        print(f"An error occurred during Google Sheets authentication: {e}")
        raise # Re-raise other exceptions


# Assuming 'spreadsheet' object is available from a previous cell that connected to Google Sheets
try:
    spreadsheet
except NameError:
    print("Spreadsheet object not found. Attempting to open spreadsheet.")
    try:
        spreadsheet = google_sheets_client.open("CRM_Capstone") # Replace with your spreadsheet name
        print("Successfully opened spreadsheet.")
    except Exception as e:
        print(f"An error occurred while opening the spreadsheet: {e}")
        raise # Re-raise the error if spreadsheet cannot be opened


# ---------- Helper functions (from cell 8AxwbCx9My_R) ----------
def strip_strings(df):
    for col in df.select_dtypes(include=["object"]).columns:
        df[col] = df[col].astype(str).str.strip().replace("nan", pd.NA)
    return df

def fix_email(email):
    if pd.isna(email):
        return email
    email = email.strip().lower()
    email = re.sub(r"[ ']", "", email)
    pattern = r'^[\w\.-]+@[\w\.-]+\.\w+$'
    return email if re.match(pattern, email) else pd.NA

def clean_region_id(df):
    region_cols = [c for c in df.columns if "Region_ID" in c]
    for col in region_cols:
        df[col] = df[col].replace("", "NA").fillna("NA")
    return df

def convert_dates(df, date_cols, to_date_only=False):
    for col in date_cols:
        if col in df.columns:
            # Use errors='coerce' to turn unparseable dates into NaT (Not a Time)
            df[col] = pd.to_datetime(df[col], errors="coerce")
            if to_date_only:
                df[col] = df[col].dt.date
    return df


def clean_all_data(data):
    # Leads
    leads = strip_strings(data["Leads"].copy()) # Use .copy() to avoid SettingWithCopyWarning
    leads = clean_region_id(leads)
    # Duplicates already handled by concat + drop_duplicates before calling this function
    if "Email" in leads.columns:
        leads["Email"] = leads["Email"].apply(fix_email)
    # Convert date columns - handle potential errors from previous stages
    leads['Created_Date'] = pd.to_datetime(leads['Created_Date'], errors='coerce')
    leads['Last_Updated'] = pd.to_datetime(leads['Last_Updated'], errors='coerce')

    # Recalculate Deal_Status_Category based on the cleaned data, especially timestamps
    # Ensure df_stage_history is available and has a Timestamp column (which should be datetime)
    if "Stage_History" in data and 'Timestamp' in data["Stage_History"].columns:
         df_stage_history_for_categorization = data["Stage_History"]
    else:
         print("Warning: df_stage_history not available or missing Timestamp for lead categorization. Categorizing ongoing deals based on Created_Date only.")
         df_stage_history_for_categorization = pd.DataFrame(columns=['Lead_ID', 'Stage', 'Timestamp']) # Empty fallback


    def categorize_deal_status(row, df_stage_history):
        # Ensure required columns exist and are not NaN
        if pd.isna(row.get('Current_Stage')):
             return 'Other'

        if row['Current_Stage'] == 'Deal Won':
            return 'Deal Won'
        elif row['Current_Stage'] == 'Deal Lost':
            return 'Deal Lost'
        elif row['Current_Stage'].startswith('Ongoing - '):
            lead_id = row['Lead_ID']

            # Ensure df_stage_history is a DataFrame and has the expected columns
            if not isinstance(df_stage_history, pd.DataFrame) or df_stage_history.empty or 'Lead_ID' not in df_stage_history.columns or 'Stage' not in df_stage_history.columns or 'Timestamp' not in df_stage_history.columns:
                 # Fallback if stage history is unusable
                 created_date_dt = pd.to_datetime(row['Created_Date'], errors='coerce')
                 if pd.isna(created_date_dt):
                     return 'Ongoing - Cold' # Cannot calculate duration without a valid date

                 duration_since_creation = datetime.now() - created_date_dt
                 two_months_in_days = 60
                 return 'Ongoing - Cold' if duration_since_creation.days > two_months_in_days else 'Ongoing - Active'


            # Filter stage history for the current lead and relevant stages
            lead_stage_history = df_stage_history[df_stage_history['Lead_ID'] == lead_id].copy()

            # Ensure Timestamp is datetime and sort
            lead_stage_history['Timestamp'] = pd.to_datetime(lead_stage_history['Timestamp'], errors='coerce')
            lead_stage_history.dropna(subset=['Timestamp'], inplace=True)
            lead_stage_history = lead_stage_history.sort_values(by='Timestamp').reset_index(drop=True)

            # Define the stages to consider for "ongoing" duration (all funnel stages before Won/Lost)
            funnel_stages_for_ongoing = [
                 "Lead Captured", "Qualified", "Contacted", "Responded",
                 "Demo Scheduled", "Demo", "Demo Completed", "Proposal Sent",
                 "Proposal", "Negotiation"
            ]

            # Find the timestamp of the latest stage that is part of the ongoing funnel progression
            latest_ongoing_stage_ts = lead_stage_history[
                lead_stage_history['Stage'].isin(funnel_stages_for_ongoing)
            ]['Timestamp'].max()


            # If no ongoing stage history found (e.g., only 'Lead Captured' history, or none), use Created_Date from Leads
            if pd.isna(latest_ongoing_stage_ts):
                 created_date_dt = pd.to_datetime(row['Created_Date'], errors='coerce')
                 if pd.isna(created_date_dt):
                     return 'Ongoing - Cold' # Cannot calculate duration without a valid date
                 latest_timestamp = created_date_dt
            else:
                 latest_timestamp = latest_ongoing_stage_ts

            # Calculate the duration from the latest relevant timestamp to now
            duration_since_last_update = datetime.now() - latest_timestamp
            two_months_in_days = 60

            if duration_since_last_update.days > two_months_in_days:
                return 'Ongoing - Cold'
            else:
                return 'Ongoing - Active'
        else:
            # For any other unexpected values in Current_Stage
            return 'Other'

    # Apply the categorization function to update the column
    leads['Deal_Status_Category'] = leads.apply(
        lambda row: categorize_deal_status(row, df_stage_history_for_categorization),
        axis=1
    )


    # Sales_Reps
    sales_reps = strip_strings(data["Sales_Reps"].copy()) # Use .copy()
    sales_reps = clean_region_id(sales_reps)
    # Duplicates already handled
    if "Email" in sales_reps.columns:
        sales_reps["Email"] = sales_reps["Email"].apply(fix_email)
    # Convert Join_Date - handle potential errors
    sales_reps['Join_Date'] = pd.to_datetime(sales_reps['Join_Date'], errors='coerce').dt.date


    # Regions
    regions = strip_strings(data["Regions"].copy()) # Use .copy()
    regions = clean_region_id(regions)
    # Duplicates already handled


    # Companies
    companies = strip_strings(data["Companies"].copy()) # Use .copy()
    companies = clean_region_id(companies)
    # Duplicates already handled


    # Stage_History
    stage_history = strip_strings(data["Stage_History"].copy()) # Use .copy()
    stage_history = clean_region_id(stage_history)
    # Duplicates already handled
    # Convert Timestamp - handle potential errors
    stage_history['Timestamp'] = pd.to_datetime(stage_history['Timestamp'], errors='coerce')
    # Fix: Remove repeated inplace=True argument
    stage_history.dropna(subset=['Timestamp'], inplace=True) # Drop rows with invalid timestamps after coercion


    # Recalculate Stage_Engagement (in Days) for the cleaned and deduplicated history
    stage_engagement_days = []
    # Ensure sorting before calculating differences
    stage_history = stage_history.sort_values(by=['Lead_ID', 'Timestamp']).reset_index(drop=True)

    for lead_id, group in stage_history.groupby('Lead_ID'):
        group = group.sort_values(by='Timestamp').reset_index(drop=True) # Ensure sorted within group
        for i in range(len(group)):
            if i == 0:
                days = 0
            else:
                # Calculate difference with the previous valid timestamp
                time_diff = group.iloc[i]['Timestamp'] - group.iloc[i-1]['Timestamp']
                days = round(time_diff.total_seconds() / (24 * 3600))
                days = max(days, 0) # Ensure non-negative days
            stage_engagement_days.append(days)

    # Assign the recalculated engagement days
    if len(stage_engagement_days) == len(stage_history):
         stage_history['Stage_Engagement (in Days)'] = stage_engagement_days
    else:
         print("Warning: Length mismatch in Stage_Engagement calculation during cleaning. Skipping assignment.")
         # Fallback or additional error handling might be needed here


    # Deals
    deals = strip_strings(data["Deals"].copy()) # Use .copy()
    deals = clean_region_id(deals)
    # Duplicates already handled
    # Convert Date columns - handle potential errors
    deals['Created_Date'] = pd.to_datetime(deals['Created_Date'], errors='coerce').dt.date
    deals['Closed_Date'] = pd.to_datetime(deals['Closed_Date'], errors='coerce').dt.date # Will be NaT for ongoing deals

    # Recalculate Total_Duration_Days for deals based on cleaned Created_Date and Closed_Date/now()
    def calculate_total_duration(row):
        created_date = row['Created_Date']
        closed_date = row['Closed_Date']
        status = row['Status']

        if pd.isna(created_date):
             return None # Cannot calculate duration without a created date

        if status in ['Deal Won', 'Deal Lost'] and pd.notna(closed_date):
             # Ensure created_date is not after closed_date
             if created_date > closed_date:
                  return (created_date - created_date).days # Duration is 0 if dates are invalidly ordered
             return (closed_date - created_date).days
        elif status.startswith('Ongoing - '):
            # Calculate duration from Created_Date to now
            return (datetime.now().date() - created_date).days
        else:
            return None # Status is neither closed nor ongoing, duration is not applicable


    deals['Total_Duration_Days'] = deals.apply(calculate_total_duration, axis=1)


    data["Leads"] = leads
    data["Sales_Reps"] = sales_reps
    data["Regions"] = regions
    data["Companies"] = companies
    data["Stage_History"] = stage_history
    data["Deals"] = deals
    return data


# ---------- Step 1: Extract from Google Sheet (Modified - relies on generation cells having already read data) ----------
# The generation cells (Lead, Company, etc.) are now responsible for reading the existing data
# and combining it with the new data.
# So, the dataframes (df_lead, df_company, etc.) already contain the combined data.
# We will put these combined dataframes into the data_from_sheets dictionary for cleaning.

# Define sheets to process - ensure these match your Google Sheet tab names
sheets_to_process = ["Leads", "Sales_Reps", "Regions", "Companies", "Stage_History", "Deals"]
data_for_cleaning = {}

# Put the combined dataframes (generated in previous cells) into the dictionary
# Ensure these dataframes are available in the kernel environment
try:
    data_for_cleaning["Leads"] = df_lead
    data_for_cleaning["Sales_Reps"] = df_sales_rep
    data_for_cleaning["Regions"] = df_region
    data_for_cleaning["Companies"] = df_company
    data_for_cleaning["Stage_History"] = df_stage_history
    data_for_cleaning["Deals"] = df_deals
    print("Combined dataframes loaded for cleaning.")
except NameError as e:
    print(f"Error: Required dataframe not found in kernel: {e}. Cannot proceed with cleaning and loading.")
    # Exit or handle the error appropriately if dataframes are missing
    raise # Re-raise to stop execution if critical data is missing


# ---------- Step 2: Deduplicate combined dataframes ----------
# Drop duplicates based on the primary key, keeping the first occurrence (manual entries prioritized)
# Assuming the ID columns are named consistently
id_columns = {
    "Leads": "Lead_ID",
    "Companies": "Company_ID",
    "Sales_Reps": "Rep_ID",
    "Regions": "Region_ID",
    "Stage_History": ["Lead_ID", "Stage", "Timestamp"], # Consider combination for history uniqueness
    "Deals": "Deal_ID"
}

deduplicated_data = {}

for sheet_name, df in data_for_cleaning.items():
    if sheet_name in id_columns:
        subset_cols = id_columns[sheet_name]
        # Ensure subset_cols are in the DataFrame columns before dropping duplicates
        valid_subset_cols = [col for col in (subset_cols if isinstance(subset_cols, list) else [subset_cols]) if col in df.columns]

        if valid_subset_cols:
            # Drop duplicates, keeping the first occurrence (manual entries first)
            df_dedup = df.drop_duplicates(subset=valid_subset_cols, keep='first').reset_index(drop=True)
            print(f"Deduplicated {sheet_name}: {len(df)} -> {len(df_dedup)}")
            deduplicated_data[sheet_name] = df_dedup
        else:
            print(f"Warning: Could not find ID columns {subset_cols} in {sheet_name}. Skipping deduplication.")
            deduplicated_data[sheet_name] = df # Keep the original if ID columns not found

    else:
        print(f"Warning: No ID column defined for {sheet_name}. Skipping deduplication.")
        deduplicated_data[sheet_name] = df # Keep original if no ID column defined


# ---------- Step 3: Transform (Clean Data) ----------
# Pass the deduplicated data to the cleaning function
cleaned_data = clean_all_data(deduplicated_data)
print("✅ Data Cleaning Complete.")

# ---------- Step 4: Load cleaned data back to Google Sheet ----------
# Use the spreadsheet object obtained from the previous successful connection
for sheet_name, df in cleaned_data.items():
    try:
        # Attempt to get the worksheet
        worksheet = spreadsheet.worksheet(sheet_name)
    except gspread.WorksheetNotFound:
        # If not found, create it (with appropriate size if df is not empty)
        rows, cols = df.shape if not df.empty else (1, 1) # Create at least 1x1 if empty
        worksheet = spreadsheet.add_worksheet(title=sheet_name, rows=rows, cols=cols)
        print(f"Created new sheet: {sheet_name}")

    try:
        # Clear existing data before writing the full, cleaned dataset
        worksheet.clear()
        # Write the cleaned dataframe back to the sheet
        if not df.empty:
            set_with_dataframe(worksheet, df) # Use set_with_dataframe to write including headers
            print(f"Successfully wrote cleaned data to sheet: {sheet_name}")
        else:
            print(f"Warning: Cleaned DataFrame for {sheet_name} is empty. Clearing sheet but writing no data.")

    except Exception as e:
        print(f"An error occurred while writing data to sheet '{sheet_name}': {e}")

print("💾 Cleaned data written back to Google Sheet.")

# ---------- Step 5: Load cleaned data as CSV files to Google Drive ----------
drive_folder_path = "/content/drive/MyDrive/CAPSTONE_FINAL/Generated_Data" # Specify your desired folder path

# Create the folder if it doesn't exist (optional, but good practice)
import os
if not os.path.exists(drive_folder_path):
    os.makedirs(drive_folder_path)
    print(f"Created Google Drive folder: {drive_folder_path}")


for sheet_name, df in cleaned_data.items():
    output_csv_path = os.path.join(drive_folder_path, f"{sheet_name}_cleaned.csv")
    try:
        # Ensure column names are valid for CSV (no leading/trailing spaces, etc.)
        df.columns = df.columns.str.strip()
        df.to_csv(output_csv_path, index=False)
        print(f"Successfully saved cleaned data to CSV in Google Drive: {output_csv_path}")
    except Exception as e:
        print(f"An error occurred while saving cleaned data to CSV for {sheet_name}: {e}")
print("💾 Cleaned data saved as CSV files in Google Drive.")


# ---------- Step 6: Load into SQLite ----------
db_path = "/content/drive/MyDrive/CAPSTONE_FINAL/CRM_Capstone_DB.sqlite"
conn = sqlite3.connect(db_path)
for sheet_name, df in cleaned_data.items():
    # Ensure column names are valid SQLite identifiers (replace spaces/special chars)
    df.columns = df.columns.str.replace(' ', '_').str.replace('[^A-Za-z0-9_]+', '', regex=False) # Use regex=False
    # Print column names for verification, especially for Stage_History
    if sheet_name == "Stage_History":
        print(f"Columns in {sheet_name} before loading to SQLite: {df.columns.tolist()}")
    df.to_sql(sheet_name, conn, if_exists="replace", index=False)
print(f"✅ Data loaded into {db_path}")


# ---------- Step 7: SQL Modeling — Correct Funnel & Joins (using SQLite connection) ----------
# Drop the view if it exists to ensure it's recreated with the latest definition
conn.execute("DROP VIEW IF EXISTS Funnel_Stages;")
print("✅ Dropped existing Funnel_Stages view (if it existed).")

# Corrected SQL to match the actual column name in the SQLite table for Stage_History
funnel_view_sql = """
CREATE VIEW Funnel_Stages AS
SELECT
    d.Deal_ID,
    l.Lead_Name,
    c.Company_Name,
    r.Region_Name,
    s.Rep_Name,
    d.Created_Date AS Deal_Created_Date, -- Renamed to clarify
    d.Closed_Date AS Deal_Closed_Date, -- Renamed to clarify
    d.Total_Duration_Days AS Deal_Total_Duration_Days,
    sh.Stage,
    CASE
        WHEN UPPER(sh.Stage) LIKE '%LEAD CAPTURED%' THEN 'Lead Captured'
        WHEN UPPER(sh.Stage) LIKE '%QUALIFIED%' THEN 'Qualified'
        WHEN UPPER(sh.Stage) LIKE '%CONTACTED%' THEN 'Contacted'
        WHEN UPPER(sh.Stage) LIKE '%DEMO SCHEDULED%' THEN 'Demo Scheduled'
        WHEN UPPER(sh.Stage) LIKE '%DEMO COMPLETED%' THEN 'Demo Completed'
        WHEN UPPER(sh.Stage) LIKE '%DEMO%' THEN 'Demo'
        WHEN UPPER(sh.Stage) LIKE '%PROPOSAL SENT%' THEN 'Proposal Sent'
        WHEN UPPER(sh.Stage) LIKE '%PROPOSAL%' THEN 'Proposal'
        WHEN UPPER(sh.Stage) LIKE '%NEGOTIATION%' THEN 'Negotiation'
        WHEN UPPER(sh.Stage) LIKE '%DEAL WON%' THEN 'Deal Won'
        WHEN UPPER(sh.Stage) LIKE '%DEAL LOST%' THEN 'Deal Lost'
        ELSE 'Other'
    END AS Funnel_Stage,
    DATE(sh.Timestamp) AS Stage_Change_Date,
    sh."Stage_Engagement_(in_Days)" AS Stage_Duration_Days, -- Corrected column name here
    l.Rep_ID
FROM Deals d
LEFT JOIN Leads l ON d.Lead_ID = l.Lead_ID
LEFT JOIN Companies c ON d.Company_ID = c.Company_ID
LEFT JOIN Regions r ON l.Region_ID = r.Region_ID
LEFT JOIN Sales_Reps s ON l.Rep_ID = s.Rep_ID
LEFT JOIN Stage_History sh ON l.Lead_ID = sh.Lead_ID;
"""
conn.executescript(funnel_view_sql)
print("✅ SQL Funnel_Stages view created.")


# ---------- Step 8: Analysis Queries (using SQLite connection) ----------
# Corrected SQL query for Avg_Duration_Per_Stage to match the actual column name
queries = {
    "Funnel_Stage_Counts": """
        SELECT Funnel_Stage, COUNT(DISTINCT Deal_ID) AS Deal_Count -- Count distinct deals per stage
        FROM Funnel_Stages
        WHERE Funnel_Stage != 'Other' -- Exclude 'Other' category if not needed for funnel analysis
        GROUP BY Funnel_Stage
        ORDER BY Deal_Count DESC;
    """,
    "Avg_Duration_Per_Stage": """
        SELECT Stage, ROUND(AVG("Stage_Engagement_(in_Days)"), 1) AS Avg_Days -- Use the correct column name
        FROM Stage_History
        WHERE "Stage_Engagement_(in_Days)" IS NOT NULL
        GROUP BY Stage
        ORDER BY Avg_Days DESC;
    """,
     "Win_Rate_By_Rep": """
        SELECT
            s.Rep_Name,
            ROUND(
              SUM(CASE WHEN fs.Funnel_Stage = 'Deal Won' THEN 1 ELSE 0 END) * 100.0
              / NULLIF(COUNT(DISTINCT fs.Deal_ID), 0), 1
            ) AS Win_Rate_Percent
        FROM Funnel_Stages fs
        LEFT JOIN Sales_Reps s ON fs.Rep_ID = s.Rep_ID
        WHERE fs.Funnel_Stage IN ('Deal Won', 'Deal Lost') -- Only consider closed deals for win rate
        GROUP BY s.Rep_Name
        ORDER BY Win_Rate_Percent DESC;
    """
}

result_dfs = {}
for name, sql in queries.items():
    try:
        result_dfs[name] = pd.read_sql_query(sql, conn)
        print(f"Successfully ran analysis query: {name}")
    except Exception as e:
        print(f"An error occurred while running query '{name}': {e}")


# ---------- Step 9: Load Analysis Results to BigQuery and Export to Excel/Google Sheets ----------
print("⬆️ Attempting to upload analysis results to BigQuery...")

# Re-using the existing BigQuery client and dataset_id
try:
    # Check if the bigquery client object exists
    bigquery_client
except NameError:
    print("BigQuery client not found. Please ensure you have authenticated with Google Cloud and initialized the BigQuery client.")
    # You might want to add authentication/client initialization here if it's not handled in a previous cell
    # Example:
    # auth.authenticate_user()
    # project_id = "your-google-cloud-project-id" # Replace with your project ID
    # bigquery_client = bigquery.Client(project=project_id)
    raise

try:
    # Create the dataset if it doesn't exist (already done for main tables, but good to ensure)
    dataset_ref = bigquery_client.dataset(dataset_id)
    bigquery_client.get_dataset(dataset_ref) # Check if dataset exists
except Exception:
    dataset = bigquery.Dataset(dataset_ref)
    dataset.location = "US"
    bigquery_client.create_dataset(dataset, timeout=30)
    print(f"Dataset '{dataset_id}' created for analysis results.")


# Define job configuration for loading analysis results
job_config_analysis = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE") # Overwrite tables


for name, df in result_dfs.items():
    # Create table ID using the analysis name
    table_id_analysis = f"{dataset_id}.{name}"

    try:
        # Ensure column names are valid BigQuery identifiers
        df.columns = df.columns.str.replace(' ', '_').str.replace('[^A-Za-z0-9_]+', '', regex=False)

        # Upload DataFrame to BigQuery
        job = bigquery_client.load_table_from_dataframe(
            df, table_id_analysis, job_config=job_config_analysis
        )

        job.result() # Wait for the job to complete

        print(f"✅ Uploaded analysis results '{name}' to BigQuery table: {table_id_analysis}")

    except Exception as e:
        print(f"❌ Error uploading analysis results to BigQuery for '{name}': {e}")
        print(f"DataFrame columns for '{name}': {df.columns.tolist()}")


# Export results to Excel and Google Sheets (existing code)
output_excel_path = "/content/drive/MyDrive/CAPSTONE_FINAL/CRM_Funnel_Stage_Results.xlsx"
with pd.ExcelWriter(output_excel_path, engine="xlsxwriter") as writer:
    for name, df in result_dfs.items():
        df.to_excel(writer, sheet_name=name, index=False)
        # Also write to Google Sheets
        try:
            worksheet = spreadsheet.worksheet(name)
        except gspread.WorksheetNotFound:
            worksheet = spreadsheet.add_worksheet(title=name, rows=df.shape[0], cols=df.shape[1])

        worksheet.clear()
        set_with_dataframe(worksheet, df)
        print(f"Successfully wrote analysis results to Google Sheet: {name}")


conn.close()
print(f"📊 Funnel stage results exported to {output_excel_path}, Google Sheets, and Google Drive CSVs.")
print("✅ You can also open the .sqlite DB in DB Browser and browse 'Funnel_Stages' to see details.")

Combined dataframes loaded for cleaning.
Deduplicated Leads: 2003 -> 2003
Deduplicated Sales_Reps: 20 -> 20
Deduplicated Regions: 6 -> 6
Deduplicated Companies: 1963 -> 1963
Deduplicated Stage_History: 37759 -> 37750
Deduplicated Deals: 2003 -> 2003
✅ Data Cleaning Complete.
Successfully wrote cleaned data to sheet: Leads
Successfully wrote cleaned data to sheet: Sales_Reps
Successfully wrote cleaned data to sheet: Regions
Successfully wrote cleaned data to sheet: Companies
Successfully wrote cleaned data to sheet: Stage_History
Successfully wrote cleaned data to sheet: Deals
💾 Cleaned data written back to Google Sheet.
Successfully saved cleaned data to CSV in Google Drive: /content/drive/MyDrive/CAPSTONE_FINAL/Generated_Data/Leads_cleaned.csv
Successfully saved cleaned data to CSV in Google Drive: /content/drive/MyDrive/CAPSTONE_FINAL/Generated_Data/Sales_Reps_cleaned.csv
Successfully saved cleaned data to CSV in Google Drive: /content/drive/MyDrive/CAPSTONE_FINAL/Generated_Data/Regi